In [5]:
import json
import joblib
import pickle
import pandas as pd
import numpy as np
import os
from datetime import datetime
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier



import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

import fairlearn

In [ ]:
#pip install xgboost

In [3]:
#pip install psycopg2-binary

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 3.3 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [14]:
df=pd.read_csv('two-years-recid.csv')#.set_index('id')
df

,id,name,sex,dob,age,race,c_jail_in,c_jail_out,c_case_number,c_offense_date,...,r_case_number,r_offense_date,r_charge_degree,r_charge_desc,is_violent_recid,vr_case_number,vr_offense_date,vr_charge_degree,vr_charge_desc,two_year_recid
0,1,steven lux,Male,1953-06-15,62,Caucasian,2013-01-05 04:35:31,2013-01-07 03:18:03,13000208CF10A,2013-01-05,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0
1,2,andre small,Male,1987-10-01,28,African-American,2013-04-02 12:18:46,2013-04-04 07:54:22,13006354MM10A,2013-04-02,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0
2,3,willie gray,Male,1959-01-12,57,African-American,2014-10-31 12:02:01,2014-10-31 01:47:05,14040148MU10A,2014-10-30,...,15043364TC20A,2015-07-23,(M2),Driving License Suspended,0,NaN,NaN,NaN,NaN,1
3,4,nickson marcellus,Male,1996-07-11,19,African-American,2014-01-23 03:19:30,2014-01-23 01:04:34,13017969CF10A,NaN,...,16000241MM20A,2016-01-04,(M1),Possess Cannabis/20 Grams Or Less,0,NaN,NaN,NaN,NaN,1
4,5,patria barnes,Female,1978-06-06,37,Other,2013-12-08 01:55:28,2013-12-09 02:00:59,13022717MM10A,2013-12-07,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6109,6110,seccunda davis,Male,1987-05-13,28,African-American,2013-08-18 07:25:24,2013-08-19 09:01:42,13015644MM10A,2013-08-18,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0
6110,6111,mark montgomery,Male,1985-11-03,30,African-American,2013-03-23 01:32:34,2013-03-28 09:37:27,13005696MM10A,2013-03-23,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0
6111,6112,erica johnson,Female,1982-06-23,33,Caucasian,2013-09-29 09:25:30,2013-09-30 09:59:37,13013661CF10A,2013-09-29,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0
6112,6113,barry williams,Male,1988-04-22,27,African-American,2013-10-30 03:40:14,2013-12-07 01:53:45,13004112MM10A,NaN,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0


In [15]:
## REMOVING ROWS FOR GOAL

In [16]:
df = df.dropna(subset=['c_jail_out'])

# Suppose df_test has some different values, so we'll modify it directly
# Convert columns to datetime if they are not already
df['c_jail_in'] = pd.to_datetime(df['c_jail_in'])
df['c_jail_out'] = pd.to_datetime(df['c_jail_out'])

df['r_offense_date'] = pd.to_datetime(df['r_offense_date'])


# Calculate the difference between 'r_offense_date' and 'c_jail_out'
df['offense_jail_duration'] = (df['r_offense_date'] - df['c_jail_out']).dt.days

# Create a new column based on your conditions
df['two_year_recid'] = ((df['is_recid'] == 1) & (df['offense_jail_duration'] <= 365 * 2)).astype(int)

#
df['is_recid']=df['two_year_recid']

/tmp/ipykernel_3513/2141686846.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['c_jail_in'] = pd.to_datetime(df['c_jail_in'])
/tmp/ipykernel_3513/2141686846.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['c_jail_out'] = pd.to_datetime(df['c_jail_out'])
/tmp/ipykernel_3513/2141686846.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydat

In [11]:
#def load_data(file):
#    df = pd.read_csv(os.path.join("data", file))
#    return df
#df_original = load_data("train.csv")
##df_new = load_data("bank_new_observations.csv")
#target='Inspection Outcome'
#df_original.head()
#
import sqlalchemy

username = 'postgres'
password = 'VuwTtzbvYMOuBUgsQykKFrvBKhWnQySt'
host_name = 'viaduct.proxy.rlwy.net'
port = 15540
db_name = 'railway'

conn_str = 'postgresql://{}:{}@{}:{}/{}'.format(username,password,host_name,port,db_name)
engine = sqlalchemy.create_engine(conn_str)
query = 'SELECT * FROM prediction;'
data_server = pd.read_sql_query(query, engine)

df = data_server.copy()
df_ = pd.DataFrame()
#for i in range(len(df['observation'])):
#    new = pd.DataFrame([json.loads(df['observation'][i])])
#    df_=pd.concat([df_,new])
#df_=df_.reset_index().drop(columns=['index'])
#df_['Inspection Outcome']=data_server['label']
#
##df_original['new'] = False
##df_['new'] = True
#concatenated_df = pd.concat([df_original, df_])
#final_df = concatenated_df.drop_duplicates(subset='observation_id', keep='last')
##final_df.set_index('observation_id', inplace = True)
#final_df
df

# Define the path where you want to save the CSV file
csv_file_path = 'prediction_data_2306.csv'

# Export the DataFrame to a CSV file
df.to_csv(csv_file_path, index=False)

print(f'DataFrame has been exported to {csv_file_path}')


DataFrame has been exported to prediction_data_2306.csv


In [17]:
df.count()

id                       5854
name                     5854
sex                      5854
dob                      5854
age                      5854
race                     5854
c_jail_in                5854
c_jail_out               5854
c_case_number            5854
c_offense_date           4983
c_arrest_date             871
c_charge_degree          5854
c_charge_desc            5848
juv_fel_count            5854
juv_misd_count           5854
juv_other_count          5854
priors_count             5854
compas_screening_date    5854
type_of_assessment       5854
decile_score             5854
score_text               5854
v_type_of_assessment     5854
v_decile_score           5854
v_score_text             5854
is_recid                 5854
r_case_number            2879
r_offense_date           2879
r_charge_degree          2879
r_charge_desc            2830
is_violent_recid         5854
vr_case_number            681
vr_offense_date           681
vr_charge_degree          681
vr_charge_

In [18]:
# Swap c_jail_in and c_jail_out where c_jail_out is before c_jail_in and the difference is less than one day
swap_mask = (df['c_jail_out'] < df['c_jail_in']) & ((df['c_jail_in'] - df['c_jail_out']).dt.days < 1)
df.loc[swap_mask, ['c_jail_in', 'c_jail_out']] = df.loc[swap_mask, ['c_jail_out', 'c_jail_in']].values


In [19]:
#non sense rows
df['invalid_jail_in_arrest'] = df['c_jail_in'] < df['c_arrest_date']
df['invalid_jail_in_offense'] = df['c_jail_in'] < df['c_offense_date']

# Remove rows where both conditions are true
df = df[~(df['invalid_jail_in_arrest'] & df['invalid_jail_in_offense'])]

# Drop the columns after filtering
df = df.drop(columns=['invalid_jail_in_arrest', 'invalid_jail_in_offense'])

/tmp/ipykernel_3513/2362044094.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['invalid_jail_in_arrest'] = df['c_jail_in'] < df['c_arrest_date']
/tmp/ipykernel_3513/2362044094.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['invalid_jail_in_offense'] = df['c_jail_in'] < df['c_offense_date']


In [20]:
df = df[df['c_jail_in'] >= '2012-01-01']

In [147]:
##FIL IN AVG
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier
#from xgboost import XGBClassifier
import joblib

# Define the categorize_charge function
# Frequency encoding
def frequency_encoding(df, column):
    freq_encoding = df[column].value_counts(normalize=True)
    df[f'{column}_freq'] = df[column].map(freq_encoding)
    return df

# Define the preprocess_data function
def preprocess_data(df):
    
    df['dob'] = pd.to_datetime(df['dob'])
    df['c_offense_date'] = pd.to_datetime(df['c_offense_date'])
    df['c_arrest_date'] = pd.to_datetime(df['c_arrest_date'])
    df['c_jail_in'] = pd.to_datetime(df['c_jail_in'])

    # Create unified_date
    df['unified_date'] = df['c_arrest_date'].combine_first(df['c_offense_date'])

    # Feature engineering
    df['age_birth'] = (pd.to_datetime('today') - df['dob']).dt.days // 365
    df['age_at_unified_date'] = (df['unified_date'] - df['dob']).dt.days // 365
    
    # Calculate averages for filling NaN values
    avg_time_offense_arrest = (df['c_arrest_date'] - df['c_offense_date']).dt.days.mean()
    avg_time_since_jail_in = (pd.to_datetime('today') - df['c_jail_in']).dt.days.mean()
    avg_time_to_jail = (df['c_jail_in'] - df['unified_date']).dt.days.mean()

    df['time_offense_arrest'] = (df['c_arrest_date'] - df['c_offense_date']).dt.days.fillna(avg_time_offense_arrest)  # Fill NaN with average
    df['time_since_jail_in'] = (pd.to_datetime('today') - df['c_jail_in']).dt.days.fillna(avg_time_since_jail_in)  # Duration in jail till today
    df['time_to_jail'] = (df['c_jail_in'] - df['unified_date']).dt.days.fillna(avg_time_to_jail)
    
    df['total_juv_crimes'] = df['juv_fel_count'] + df['juv_misd_count'] + df['juv_other_count']
    df['total_adult_crimes'] = df['priors_count'] - df['total_juv_crimes']

    
    bins = [0, 1, 3, 5, 10, np.inf]
    labels = ['0', '1-2', '3-4', '5-9', '10+']
    df['priors_count_bin'] = pd.cut(df['priors_count'], bins=bins, labels=labels, right=False)

    
    # Apply frequency encoding and group less frequent categories
    df = frequency_encoding(df, 'c_charge_desc')

    # Create age bins and handle missing values
    bins = [18, 25, 35, 45, 55, 65, 100]
    labels = ['18-25', '26-35', '36-45', '46-55', '56-65', '66+']
    df['age_group'] = pd.cut(df['age_birth'], bins=bins, labels=labels, right=False)
    df['age_group'] = df['age_group'].cat.add_categories('missing_value').fillna('missing_value')
    
    # Extract more granular date features
    df['offense_month'] = df['c_offense_date'].dt.month.fillna(df['c_offense_date'].dt.month.mean())
    df['offense_day_of_week'] = df['c_offense_date'].dt.dayofweek.fillna(df['c_offense_date'].dt.dayofweek.mean())
    df['arrest_month'] = df['c_arrest_date'].dt.month.fillna(df['c_arrest_date'].dt.month.mean())
    df['arrest_day_of_week'] = df['c_arrest_date'].dt.dayofweek.fillna(df['c_arrest_date'].dt.dayofweek.mean())
    df['sex_race_interaction'] = df['sex'] + '_' + df['race']
    return df

# Load data (ensure df is loaded before this step)
# df = pd.read_csv('your_data.csv')  # Example loading method

# Apply preprocessing to the entire dataset
df = preprocess_data(df)

# Split data into train and test sets
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

# Define features
all_features = [
    "sex", 
    "race", 
    #"juv_fel_count", 
    #"juv_misd_count", 
    #"juv_other_count",
    "total_adult_crimes", 
    "c_charge_degree", 
   # "c_charge_desc", 
    "c_charge_desc_freq",
    "age_at_unified_date", 
    "total_juv_crimes",
    "offense_month", 
    "offense_day_of_week", 
    "arrest_month", 
    "arrest_day_of_week",
    "age_group",
    "priors_count_bin"
]

numerical_features = [
   # "juv_fel_count", 
    #"juv_misd_count", 
    #"juv_other_count", 
    "total_adult_crimes",
    #"time_offense_arrest", 
    #"time_since_jail_in", 
    "total_juv_crimes",
    "offense_month", 
    "offense_day_of_week", 
    "arrest_month", 
    "arrest_day_of_week",
    "age_at_unified_date", 
    #"age_birth",
    "c_charge_desc_freq",
    #"priors_count"
]

categorical_features = [
    "sex", 
    "race", 
    "c_charge_degree", 
   # "c_charge_desc",
    "age_group",
    "priors_count_bin",
]

target = 'is_recid'

# Define preprocessing pipelines
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing_value')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Use the best model parameters found earlier
best_model_params = {
    'learning_rate': 0.01,
    'max_depth': 3,
    'n_estimators': 300,
    'subsample': 0.8,
    'random_state': 42
}

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', GradientBoostingClassifier(**best_model_params))
])


# Train the updated pipeline
X_train = df_train[all_features]
y_train = df_train[target]
pipeline.fit(X_train, y_train)

# Evaluate the updated pipeline on the test set
X_test = df_test[all_features]
y_test = df_test[target]
y_pred = pipeline.predict(X_test)
y_proba = pipeline.predict_proba(X_test)[:, 1]

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_proba)
conf_matrix = confusion_matrix(y_test, y_pred)

# Print evaluation metrics
print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1-Score: {f1:.2f}')
print(f'ROC-AUC: {roc_auc:.2f}')
print(f'Confusion Matrix:\n{conf_matrix}')

#####FAIRNESS#######
# Fairness and Bias Metrics
from fairlearn.metrics import (
    demographic_parity_ratio,
    equalized_odds_ratio,
    true_positive_rate_difference,
    false_positive_rate_difference,
    selection_rate_difference
)
import numpy as np

# Function to calculate fairness metrics for a given feature

# Fairness metrics
for feature in ['sex', 'race', 'age_group','priors_count_bin']:
    print(f"\nFairness metrics for {feature}:")
    disparate_impact = demographic_parity_ratio(y_true=y_test, y_pred=y_pred, sensitive_features=X_test[feature])
    equalized_odds = equalized_odds_ratio(y_true=y_test, y_pred=y_pred, sensitive_features=X_test[feature])
    tpr_diff = true_positive_rate_difference(y_true=y_test, y_pred=y_pred, sensitive_features=X_test[feature])
    fpr_diff = false_positive_rate_difference(y_true=y_test, y_pred=y_pred, sensitive_features=X_test[feature])
    selection_rate_diff = selection_rate_difference(y_true=y_test, y_pred=y_pred, sensitive_features=X_test[feature])
    print(f"Disparate Impact Ratio: {disparate_impact:.2f}")
    print(f"Equalized Odds Ratio: {equalized_odds:.2f}")
    print(f"True Positive Rate Difference: {tpr_diff:.2f}")
    print(f"False Positive Rate Difference: {fpr_diff:.2f}")
    print(f"Selection Rate Difference: {selection_rate_diff:.2f}")

# Disparate impact ratios by group for each sensitive feature
def calculate_disparate_impact(y_true, y_pred, X_test, column):
    disparate_impact_ratios = {}
    overall_favorable_outcomes = y_pred.mean()
    for value in X_test[column].unique():
        mask = X_test[column] == value
        favorable_outcomes = y_pred[mask].mean()
        if overall_favorable_outcomes == 0:
            disparate_impact_ratios[value] = np.nan
        else:
            disparate_impact_ratios[value] = favorable_outcomes / overall_favorable_outcomes
    return disparate_impact_ratios

for feature in ['sex', 'race', 'age_group', 'priors_count_bin']:
    disparate_impact_by_group = calculate_disparate_impact(y_test, y_pred, X_test, feature)
    print(f"\nDisparate Impact Ratios by {feature.capitalize()}:")
    for group, ratio in disparate_impact_by_group.items():
        print(f"{group}: {ratio:.2f}")


Accuracy: 0.69
Precision: 0.68
Recall: 0.60
F1-Score: 0.64
ROC-AUC: 0.74
Confusion Matrix:
[[495 149]
 [211 316]]

Fairness metrics for sex:
Disparate Impact Ratio: 0.55
Equalized Odds Ratio: 0.45
True Positive Rate Difference: 0.15
False Positive Rate Difference: 0.15
Selection Rate Difference: 0.19

Fairness metrics for race:
Disparate Impact Ratio: 0.22
Equalized Odds Ratio: 0.00
True Positive Rate Difference: 1.00
False Positive Rate Difference: 0.30
Selection Rate Difference: 0.58

Fairness metrics for age_group:
Disparate Impact Ratio: 0.00
Equalized Odds Ratio: 0.00
True Positive Rate Difference: 0.68
False Positive Rate Difference: 0.41
Selection Rate Difference: 0.56

Fairness metrics for priors_count_bin:
Disparate Impact Ratio: 0.07
Equalized Odds Ratio: 0.04
True Positive Rate Difference: 0.85
False Positive Rate Difference: 0.96
Selection Rate Difference: 0.93

Disparate Impact Ratios by Sex:
Male: 1.10
Female: 0.61

Disparate Impact Ratios by Race:
African-American: 1.26


In [155]:
import pandas as pd
from fairlearn.metrics import (
    demographic_parity_ratio, equalized_odds_ratio,
    true_positive_rate_difference, false_positive_rate_difference,
    selection_rate_difference
)

# Function to calculate fairness metrics for a given feature
def calculate_fairness_metrics(y_test, y_pred, X_test, feature):
    metrics = {
        'Feature': feature,
        'Disparate Impact Ratio': demographic_parity_ratio(y_true=y_test, y_pred=y_pred, sensitive_features=X_test[feature]),
        'Equalized Odds Ratio': equalized_odds_ratio(y_true=y_test, y_pred=y_pred, sensitive_features=X_test[feature]),
        'TPR Difference': true_positive_rate_difference(y_true=y_test, y_pred=y_pred, sensitive_features=X_test[feature]),
        'FPR Difference': false_positive_rate_difference(y_true=y_test, y_pred=y_pred, sensitive_features=X_test[feature]),
        'Selection Rate Difference': selection_rate_difference(y_true=y_test, y_pred=y_pred, sensitive_features=X_test[feature])
    }
    return metrics

# Function to calculate disparate impact ratios by group for each sensitive feature
def calculate_disparate_impact(y_true, y_pred, X_test, column):
    disparate_impact_ratios = {}
    overall_favorable_outcomes = y_pred.mean()
    for value in X_test[column].unique():
        mask = X_test[column] == value
        favorable_outcomes = y_pred[mask].mean()
        if overall_favorable_outcomes == 0:
            disparate_impact_ratios[value] = np.nan
        else:
            disparate_impact_ratios[value] = favorable_outcomes / overall_favorable_outcomes
    return disparate_impact_ratios

# Create lists to store fairness metrics and disparate impact ratios
fairness_metrics_list = []
disparate_impact_list = []

# Calculate fairness metrics for each feature
features_to_check = ['sex', 'race', 'age_group', 'priors_count_bin']
for feature in features_to_check:
    metrics = calculate_fairness_metrics(y_test, y_pred, X_test, feature)
    fairness_metrics_list.append(metrics)

    # Calculate and store disparate impact ratios by group
    disparate_impact_by_group = calculate_disparate_impact(y_test, y_pred, X_test, feature)
    for group, ratio in disparate_impact_by_group.items():
        disparate_impact_list.append({'Feature': feature, 'Group': group, 'Disparate Impact Ratio': ratio})

# Convert lists to DataFrames
fairness_metrics_df = pd.DataFrame(fairness_metrics_list)
disparate_impact_df = pd.DataFrame(disparate_impact_list)

# Reorder columns to have 'Feature' as the first column
fairness_metrics_df = fairness_metrics_df[['Feature', 'Disparate Impact Ratio', 'Equalized Odds Ratio', 'TPR Difference', 'FPR Difference', 'Selection Rate Difference']]
disparate_impact_df = disparate_impact_df[['Feature', 'Group', 'Disparate Impact Ratio']]

# Format and print the DataFrames
print("Fairness Metrics:")
print(fairness_metrics_df.round(2).to_string(index=False))

print("\nDisparate Impact Ratios by Group:")
print(disparate_impact_df.round(2).to_string(index=False))


Fairness Metrics:
         Feature  Disparate Impact Ratio  Equalized Odds Ratio  TPR Difference  FPR Difference  Selection Rate Difference
             sex                    0.55                  0.45            0.15            0.15                       0.19
            race                    0.22                  0.00            1.00            0.30                       0.58
       age_group                    0.00                  0.00            0.68            0.41                       0.56
priors_count_bin                    0.07                  0.04            0.85            0.96                       0.93

Disparate Impact Ratios by Group:
         Feature            Group  Disparate Impact Ratio
             sex             Male                    1.10
             sex           Female                    0.61
            race African-American                    1.26
            race            Asian                    0.42
            race        Caucasian              

## SMOTE

In [157]:

import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier
#from xgboost import XGBClassifier
import joblib

# Define the categorize_charge function
# Frequency encoding
def frequency_encoding(df, column):
    freq_encoding = df[column].value_counts(normalize=True)
    df[f'{column}_freq'] = df[column].map(freq_encoding)
    return df

# Define the preprocess_data function
def preprocess_data(df):
    
    df['dob'] = pd.to_datetime(df['dob'])
    df['c_offense_date'] = pd.to_datetime(df['c_offense_date'])
    df['c_arrest_date'] = pd.to_datetime(df['c_arrest_date'])
    df['c_jail_in'] = pd.to_datetime(df['c_jail_in'])

    # Create unified_date
    df['unified_date'] = df['c_arrest_date'].combine_first(df['c_offense_date'])

    # Feature engineering
    df['age_birth'] = (pd.to_datetime('today') - df['dob']).dt.days // 365
    df['age_at_unified_date'] = (df['unified_date'] - df['dob']).dt.days // 365
    
    # Calculate averages for filling NaN values
    avg_time_offense_arrest = (df['c_arrest_date'] - df['c_offense_date']).dt.days.mean()
    avg_time_since_jail_in = (pd.to_datetime('today') - df['c_jail_in']).dt.days.mean()
    avg_time_to_jail = (df['c_jail_in'] - df['unified_date']).dt.days.mean()

    df['time_offense_arrest'] = (df['c_arrest_date'] - df['c_offense_date']).dt.days.fillna(avg_time_offense_arrest)  # Fill NaN with average
    df['time_since_jail_in'] = (pd.to_datetime('today') - df['c_jail_in']).dt.days.fillna(avg_time_since_jail_in)  # Duration in jail till today
    df['time_to_jail'] = (df['c_jail_in'] - df['unified_date']).dt.days.fillna(avg_time_to_jail)
    
    df['total_juv_crimes'] = df['juv_fel_count'] + df['juv_misd_count'] + df['juv_other_count']
    df['total_adult_crimes'] = df['priors_count'] - df['total_juv_crimes']

    
    bins = [0, 1, 3, 5, 10, np.inf]
    labels = ['0', '1-2', '3-4', '5-9', '10+']
    df['priors_count_bin'] = pd.cut(df['priors_count'], bins=bins, labels=labels, right=False)

    
    # Apply frequency encoding and group less frequent categories
    df = frequency_encoding(df, 'c_charge_desc')

    # Create age bins and handle missing values
    bins = [18, 25, 35, 45, 55, 65, 100]
    labels = ['18-25', '26-35', '36-45', '46-55', '56-65', '66+']
    df['age_group'] = pd.cut(df['age_birth'], bins=bins, labels=labels, right=False)
    df['age_group'] = df['age_group'].cat.add_categories('missing_value').fillna('missing_value')
    
    # Extract more granular date features
    df['offense_month'] = df['c_offense_date'].dt.month.fillna(df['c_offense_date'].dt.month.mean())
    df['offense_day_of_week'] = df['c_offense_date'].dt.dayofweek.fillna(df['c_offense_date'].dt.dayofweek.mean())
    df['arrest_month'] = df['c_arrest_date'].dt.month.fillna(df['c_arrest_date'].dt.month.mean())
    df['arrest_day_of_week'] = df['c_arrest_date'].dt.dayofweek.fillna(df['c_arrest_date'].dt.dayofweek.mean())
    df['sex_race_interaction'] = df['sex'] + '_' + df['race']
    
    # Binning total_adult_crimes
    df['total_adult_crimes'] = df['total_adult_crimes'].apply(lambda x: max(0, x)).astype(float)
    bins = [0, 1, 3, 5, 10, np.inf]
    labels = ['0', '1-2', '3-4', '5-9', '10+']
    df['total_adult_crimes_bin'] = pd.cut(df['total_adult_crimes'], bins=bins, labels=labels, right=False)

    return df

# Load data (ensure df is loaded before this step)
# df = pd.read_csv('your_data.csv')  # Example loading method

# Apply preprocessing to the entire dataset
df = preprocess_data(df)

# Split data into train and test sets
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

# Define features
all_features = [
    "sex", 
    "race", 
    #"juv_fel_count", 
    #"juv_misd_count", 
    #"juv_other_count",
    "total_adult_crimes", 
    "c_charge_degree", 
   # "c_charge_desc", 
    "c_charge_desc_freq",
    "age_at_unified_date", 
    "total_juv_crimes",
    "offense_month", 
    "offense_day_of_week", 
    "arrest_month", 
    "arrest_day_of_week",
    "age_group",
    "priors_count_bin"
]

numerical_features = [
   # "juv_fel_count", 
    #"juv_misd_count", 
    #"juv_other_count", 
    "total_adult_crimes",
    #"time_offense_arrest", 
    #"time_since_jail_in", 
    "total_juv_crimes",
    "offense_month", 
    "offense_day_of_week", 
    "arrest_month", 
    "arrest_day_of_week",
    "age_at_unified_date", 
    #"age_birth",
    "c_charge_desc_freq",
    #"priors_count"
]

categorical_features = [
    "sex", 
    "race", 
    "c_charge_degree", 
   # "c_charge_desc",
    "age_group",
    "priors_count_bin",
]

target = 'is_recid'

# Define preprocessing pipelines
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing_value')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Use the best model parameters found earlier
best_model_params = {
    'learning_rate': 0.01,
    'max_depth': 3,
    'n_estimators': 300,
    'subsample': 0.8,
    'random_state': 42
}

pipeline = ImbPipeline(steps=[
    ('preprocessor', preprocessor),
    ('smote', SMOTE(random_state=42)),
    ('classifier', GradientBoostingClassifier(**best_model_params))
])


# Train the updated pipeline
X_train = df_train[all_features]
y_train = df_train[target]
pipeline.fit(X_train, y_train)

# Evaluate the updated pipeline on the test set
X_test = df_test[all_features]
y_test = df_test[target]
y_pred = pipeline.predict(X_test)
y_proba = pipeline.predict_proba(X_test)[:, 1]

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_proba)
conf_matrix = confusion_matrix(y_test, y_pred)

# Print evaluation metrics
print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1-Score: {f1:.2f}')
print(f'ROC-AUC: {roc_auc:.2f}')
print(f'Confusion Matrix:\n{conf_matrix}')

#####FAIRNESS#######
import pandas as pd
from fairlearn.metrics import (
    demographic_parity_ratio, equalized_odds_ratio,
    true_positive_rate_difference, false_positive_rate_difference,
    selection_rate_difference
)

# Function to calculate fairness metrics for a given feature
def calculate_fairness_metrics(y_test, y_pred, X_test, feature):
    metrics = {
        'Feature': feature,
        'Disparate Impact Ratio': demographic_parity_ratio(y_true=y_test, y_pred=y_pred, sensitive_features=X_test[feature]),
        'Equalized Odds Ratio': equalized_odds_ratio(y_true=y_test, y_pred=y_pred, sensitive_features=X_test[feature]),
        'TPR Difference': true_positive_rate_difference(y_true=y_test, y_pred=y_pred, sensitive_features=X_test[feature]),
        'FPR Difference': false_positive_rate_difference(y_true=y_test, y_pred=y_pred, sensitive_features=X_test[feature]),
        'Selection Rate Difference': selection_rate_difference(y_true=y_test, y_pred=y_pred, sensitive_features=X_test[feature])
    }
    return metrics

# Function to calculate disparate impact ratios by group for each sensitive feature
def calculate_disparate_impact(y_true, y_pred, X_test, column):
    disparate_impact_ratios = {}
    overall_favorable_outcomes = y_pred.mean()
    for value in X_test[column].unique():
        mask = X_test[column] == value
        favorable_outcomes = y_pred[mask].mean()
        if overall_favorable_outcomes == 0:
            disparate_impact_ratios[value] = np.nan
        else:
            disparate_impact_ratios[value] = favorable_outcomes / overall_favorable_outcomes
    return disparate_impact_ratios

# Create lists to store fairness metrics and disparate impact ratios
fairness_metrics_list = []
disparate_impact_list = []

# Calculate fairness metrics for each feature
features_to_check = ['sex', 'race', 'age_group', 'priors_count_bin']
for feature in features_to_check:
    metrics = calculate_fairness_metrics(y_test, y_pred, X_test, feature)
    fairness_metrics_list.append(metrics)

    # Calculate and store disparate impact ratios by group
    disparate_impact_by_group = calculate_disparate_impact(y_test, y_pred, X_test, feature)
    for group, ratio in disparate_impact_by_group.items():
        disparate_impact_list.append({'Feature': feature, 'Group': group, 'Disparate Impact Ratio': ratio})

# Convert lists to DataFrames
fairness_metrics_df = pd.DataFrame(fairness_metrics_list)
disparate_impact_df = pd.DataFrame(disparate_impact_list)

# Reorder columns to have 'Feature' as the first column
fairness_metrics_df = fairness_metrics_df[['Feature', 'Disparate Impact Ratio', 'Equalized Odds Ratio', 'TPR Difference', 'FPR Difference', 'Selection Rate Difference']]
disparate_impact_df = disparate_impact_df[['Feature', 'Group', 'Disparate Impact Ratio']]

# Format and print the DataFrames
print("Fairness Metrics:")
print(fairness_metrics_df.round(2).to_string(index=False))

print("\nDisparate Impact Ratios by Group:")
print(disparate_impact_df.round(2).to_string(index=False))


Accuracy: 0.67
Precision: 0.63
Recall: 0.65
F1-Score: 0.64
ROC-AUC: 0.74
Confusion Matrix:
[[444 200]
 [182 345]]
Fairness Metrics:
         Feature  Disparate Impact Ratio  Equalized Odds Ratio  TPR Difference  FPR Difference  Selection Rate Difference
             sex                    0.55                  0.48            0.19            0.19                       0.23
            race                    0.22                  0.00            1.00            0.43                       0.58
       age_group                    0.00                  0.00            0.77            0.53                       0.66
priors_count_bin                    0.13                  0.07            0.71            0.93                       0.86

Disparate Impact Ratios by Group:
         Feature            Group  Disparate Impact Ratio
             sex             Male                    1.10
             sex           Female                    0.61
            race African-American                

##  Re-train with fairness constraints

In [104]:
#pip install aif360
#pip install tensorflow

In [159]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
)
from fairlearn.reductions import ExponentiatedGradient, DemographicParity
from fairlearn.metrics import (
    demographic_parity_ratio, equalized_odds_ratio,
    true_positive_rate_difference, false_positive_rate_difference,
    selection_rate_difference
)

# Define the categorize_charge function
def frequency_encoding(df, column):
    freq_encoding = df[column].value_counts(normalize=True)
    df[f'{column}_freq'] = df[column].map(freq_encoding)
    return df

# Define the preprocess_data function
def preprocess_data(df):
    df['dob'] = pd.to_datetime(df['dob'])
    df['c_offense_date'] = pd.to_datetime(df['c_offense_date'])
    df['c_arrest_date'] = pd.to_datetime(df['c_arrest_date'])
    df['c_jail_in'] = pd.to_datetime(df['c_jail_in'])

    # Create unified_date
    df['unified_date'] = df['c_arrest_date'].combine_first(df['c_offense_date'])

    # Feature engineering
    df['age_birth'] = (pd.to_datetime('today') - df['dob']).dt.days // 365
    df['age_at_unified_date'] = (df['unified_date'] - df['dob']).dt.days // 365

    # Calculate averages for filling NaN values
    avg_time_offense_arrest = (df['c_arrest_date'] - df['c_offense_date']).dt.days.mean()
    avg_time_since_jail_in = (pd.to_datetime('today') - df['c_jail_in']).dt.days.mean()
    avg_time_to_jail = (df['c_jail_in'] - df['unified_date']).dt.days.mean()

    df['time_offense_arrest'] = (df['c_arrest_date'] - df['c_offense_date']).dt.days.fillna(avg_time_offense_arrest)
    df['time_since_jail_in'] = (pd.to_datetime('today') - df['c_jail_in']).dt.days.fillna(avg_time_since_jail_in)
    df['time_to_jail'] = (df['c_jail_in'] - df['unified_date']).dt.days.fillna(avg_time_to_jail)
    
    df['total_juv_crimes'] = df['juv_fel_count'] + df['juv_misd_count'] + df['juv_other_count']
    df['total_adult_crimes'] = df['priors_count'] - df['total_juv_crimes']

    # Apply frequency encoding and group less frequent categories
    df = frequency_encoding(df, 'c_charge_desc')

    # Create age bins and handle missing values
    bins = [18, 25, 35, 45, 55, 65, 100]
    labels = ['18-25', '26-35', '36-45', '46-55', '56-65', '66+']
    df['age_group'] = pd.cut(df['age_birth'], bins=bins, labels=labels, right=False)
    df['age_group'] = df['age_group'].cat.add_categories('missing_value').fillna('missing_value')
    
    # Extract more granular date features
    df['offense_month'] = df['c_offense_date'].dt.month.fillna(df['c_offense_date'].dt.month.mean())
    df['offense_day_of_week'] = df['c_offense_date'].dt.dayofweek.fillna(df['c_offense_date'].dt.dayofweek.mean())
    df['arrest_month'] = df['c_arrest_date'].dt.month.fillna(df['c_arrest_date'].dt.month.mean())
    df['arrest_day_of_week'] = df['c_arrest_date'].dt.dayofweek.fillna(df['c_arrest_date'].dt.dayofweek.mean())

    # Binning total_adult_crimes
    df['total_adult_crimes'] = df['total_adult_crimes'].apply(lambda x: max(0, x)).astype(float)
    bins = [0, 1, 3, 5, 10, np.inf]
    labels = ['0', '1-2', '3-4', '5-9', '10+']
    df['priors_count_bin'] = pd.cut(df['priors_count'], bins=bins, labels=labels, right=False)

    return df

# Load data and preprocess

df = preprocess_data(df)

# Split data into train and test sets
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

# Define features
all_features = [
    "sex", "race", "priors_count_bin", "c_charge_degree",
    "c_charge_desc_freq", "age_at_unified_date", "total_juv_crimes",
    "offense_month", "offense_day_of_week", "arrest_month", "arrest_day_of_week"
]

numerical_features = [
    "total_juv_crimes", "offense_month", "offense_day_of_week",
    "arrest_month", "arrest_day_of_week", "age_at_unified_date",
    "c_charge_desc_freq"
]

categorical_features = ["sex", "race", "c_charge_degree", "priors_count_bin"]

target = 'is_recid'

# Define preprocessing pipelines
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing_value')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Use the best model parameters found earlier
best_model_params = {
    'learning_rate': 0.01,
    'max_depth': 3,
    'n_estimators': 300,
    'subsample': 0.8,
    'random_state': 42
}

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', GradientBoostingClassifier(**best_model_params))
])

# Train the updated pipeline without fairness constraints first
X_train = df_train[all_features]
y_train = df_train[target]
pipeline.fit(X_train, y_train)

# Transform the features
X_train_transformed = pipeline.named_steps['preprocessor'].transform(X_train)
X_test_transformed = pipeline.named_steps['preprocessor'].transform(df_test[all_features])

# Re-train with fairness constraints
estimator = GradientBoostingClassifier(**best_model_params)
constraint = DemographicParity()
mitigator = ExponentiatedGradient(estimator, constraints=constraint)
mitigator.fit(X_train_transformed, y_train, sensitive_features=X_train['sex'])

# Evaluate the updated pipeline on the test set
y_test = df_test[target]
y_pred = mitigator.predict(X_test_transformed)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

# Print evaluation metrics
print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1-Score: {f1:.2f}')
print(f'ROC-AUC: {roc_auc:.2f}')
print(f'Confusion Matrix:\n{conf_matrix}')

#####FAIRNESS#######
import pandas as pd
from fairlearn.metrics import (
    demographic_parity_ratio, equalized_odds_ratio,
    true_positive_rate_difference, false_positive_rate_difference,
    selection_rate_difference
)

# Function to calculate fairness metrics for a given feature
def calculate_fairness_metrics(y_test, y_pred, X_test, feature):
    metrics = {
        'Feature': feature,
        'Disparate Impact Ratio': demographic_parity_ratio(y_true=y_test, y_pred=y_pred, sensitive_features=X_test[feature]),
        'Equalized Odds Ratio': equalized_odds_ratio(y_true=y_test, y_pred=y_pred, sensitive_features=X_test[feature]),
        'TPR Difference': true_positive_rate_difference(y_true=y_test, y_pred=y_pred, sensitive_features=X_test[feature]),
        'FPR Difference': false_positive_rate_difference(y_true=y_test, y_pred=y_pred, sensitive_features=X_test[feature]),
        'Selection Rate Difference': selection_rate_difference(y_true=y_test, y_pred=y_pred, sensitive_features=X_test[feature])
    }
    return metrics

# Function to calculate disparate impact ratios by group for each sensitive feature
def calculate_disparate_impact(y_true, y_pred, X_test, column):
    disparate_impact_ratios = {}
    overall_favorable_outcomes = y_pred.mean()
    for value in X_test[column].unique():
        mask = X_test[column] == value
        favorable_outcomes = y_pred[mask].mean()
        if overall_favorable_outcomes == 0:
            disparate_impact_ratios[value] = np.nan
        else:
            disparate_impact_ratios[value] = favorable_outcomes / overall_favorable_outcomes
    return disparate_impact_ratios

# Create lists to store fairness metrics and disparate impact ratios
fairness_metrics_list = []
disparate_impact_list = []

# Calculate fairness metrics for each feature
features_to_check = ['sex', 'race', 'age_group', 'priors_count_bin']
for feature in features_to_check:
    metrics = calculate_fairness_metrics(y_test, y_pred, X_test, feature)
    fairness_metrics_list.append(metrics)

    # Calculate and store disparate impact ratios by group
    disparate_impact_by_group = calculate_disparate_impact(y_test, y_pred, X_test, feature)
    for group, ratio in disparate_impact_by_group.items():
        disparate_impact_list.append({'Feature': feature, 'Group': group, 'Disparate Impact Ratio': ratio})

# Convert lists to DataFrames
fairness_metrics_df = pd.DataFrame(fairness_metrics_list)
disparate_impact_df = pd.DataFrame(disparate_impact_list)

# Reorder columns to have 'Feature' as the first column
fairness_metrics_df = fairness_metrics_df[['Feature', 'Disparate Impact Ratio', 'Equalized Odds Ratio', 'TPR Difference', 'FPR Difference', 'Selection Rate Difference']]
disparate_impact_df = disparate_impact_df[['Feature', 'Group', 'Disparate Impact Ratio']]

# Format and print the DataFrames
print("Fairness Metrics:")
print(fairness_metrics_df.round(2).to_string(index=False))

print("\nDisparate Impact Ratios by Group:")
print(disparate_impact_df.round(2).to_string(index=False))


Accuracy: 0.68
Precision: 0.67
Recall: 0.58
F1-Score: 0.62
ROC-AUC: 0.67
Confusion Matrix:
[[492 152]
 [220 307]]
Fairness Metrics:
         Feature  Disparate Impact Ratio  Equalized Odds Ratio  TPR Difference  FPR Difference  Selection Rate Difference
             sex                    0.94                  0.92            0.05            0.01                       0.02
            race                    0.22                  0.00            1.00            0.33                       0.58
       age_group                    0.00                  0.00            0.69            0.43                       0.56
priors_count_bin                    0.09                  0.07            0.80            0.93                       0.88

Disparate Impact Ratios by Group:
         Feature            Group  Disparate Impact Ratio
             sex             Male                    1.01
             sex           Female                    0.96
            race African-American                

## Reweighing Technique

In [163]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from fairlearn.metrics import (
    demographic_parity_ratio, equalized_odds_ratio,
    true_positive_rate_difference, false_positive_rate_difference,
    selection_rate_difference
)

# Define preprocess_data function to include binning for total_adult_crimes
def preprocess_data(df):
    df['dob'] = pd.to_datetime(df['dob'])
    df['c_offense_date'] = pd.to_datetime(df['c_offense_date'])
    df['c_arrest_date'] = pd.to_datetime(df['c_arrest_date'])
    df['c_jail_in'] = pd.to_datetime(df['c_jail_in'])

    df['unified_date'] = df['c_arrest_date'].combine_first(df['c_offense_date'])

    df['age_birth'] = (pd.to_datetime('today') - df['dob']).dt.days // 365
    df['age_at_unified_date'] = (df['unified_date'] - df['dob']).dt.days // 365
    
    avg_time_offense_arrest = (df['c_arrest_date'] - df['c_offense_date']).dt.days.mean()
    avg_time_since_jail_in = (pd.to_datetime('today') - df['c_jail_in']).dt.days.mean()
    avg_time_to_jail = (df['c_jail_in'] - df['unified_date']).dt.days.mean()

    df['time_offense_arrest'] = (df['c_arrest_date'] - df['c_offense_date']).dt.days.fillna(avg_time_offense_arrest)
    df['time_since_jail_in'] = (pd.to_datetime('today') - df['c_jail_in']).dt.days.fillna(avg_time_since_jail_in)
    df['time_to_jail'] = (df['c_jail_in'] - df['unified_date']).dt.days.fillna(avg_time_to_jail)
    
    df['total_juv_crimes'] = df['juv_fel_count'] + df['juv_misd_count'] + df['juv_other_count']
    df['total_adult_crimes'] = df['priors_count'] - df['total_juv_crimes']
    
    # Apply frequency encoding
    df['c_charge_desc_freq'] = df['c_charge_desc'].map(df['c_charge_desc'].value_counts(normalize=True))

    # Create age bins and handle missing values
    bins = [18, 25, 35, 45, 55, 65, 100]
    labels = ['18-25', '26-35', '36-45', '46-55', '56-65', '66+']
    df['age_group'] = pd.cut(df['age_birth'], bins=bins, labels=labels, right=False)
    df['age_group'] = df['age_group'].cat.add_categories('missing_value').fillna('missing_value')
    
    
    # Create bins for total_adult_crimes
    bins = [-np.inf, 0, 1, 3, 5, 10, np.inf]
    labels = ['<0', '0', '1-2', '3-4', '5-9', '10+']
    df['total_adult_crimes_bin'] = pd.cut(df['total_adult_crimes'], bins=bins, labels=labels)

    # Encode categorical variables
    df['sex_encoded'] = df['sex'].map({'Male': 1, 'Female': 0})
    df['race_encoded'] = df['race'].astype('category').cat.codes
    df['priors_count_bin'] = pd.cut(df['priors_count'], bins=bins, labels=labels, right=False)

    return df

# Assuming df is already loaded and preprocessed
# Apply preprocessing to the entire dataset
df = preprocess_data(df)

# Split data into train and test sets
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

# Define features
all_features = [
    #"sex", 
    "race", 
    "priors_count_bin", 
    "c_charge_degree", 
    "c_charge_desc_freq",
    "age_at_unified_date", 
    "total_juv_crimes",
    "offense_month", 
    "offense_day_of_week", 
    "arrest_month", 
    "arrest_day_of_week",
    "age_group"
]

numerical_features = [
    "total_juv_crimes",
    "offense_month", 
    "offense_day_of_week", 
    "arrest_month", 
    "arrest_day_of_week",
    "age_at_unified_date", 
    "c_charge_desc_freq"
]

categorical_features = [
    #"sex", 
    "race", 
    "priors_count_bin", 
    "c_charge_degree",
    "age_group"
]

target = 'is_recid'

# Define preprocessing pipelines
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing_value')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Use the best model parameters found earlier
best_model_params = {
    'learning_rate': 0.01,
    'max_depth': 3,
    'n_estimators': 300,
    'subsample': 0.8,
    'random_state': 42
}

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', GradientBoostingClassifier(**best_model_params))
])

# Compute sample weights for multiple sensitive attributes
def compute_combined_sample_weights(df, sensitive_features):
    sample_weights = np.ones(len(df))
    for feature in sensitive_features:
        counts = df[feature].value_counts()
        weights = 1 / counts
        feature_weights = df[feature].map(weights)
        sample_weights *= feature_weights
    
    # Normalize weights
    sample_weights /= np.sum(sample_weights) / len(sample_weights)
    return sample_weights

# Compute combined sample weights
sensitive_features = ['race_encoded']
sample_weights = compute_combined_sample_weights(df_train, sensitive_features)

# Fit the model using manually computed sample weights
X_train = df_train[all_features]
y_train = df_train[target]
pipeline.fit(X_train, y_train, classifier__sample_weight=sample_weights)

# Evaluate the updated pipeline on the test set
X_test = df_test[all_features]
y_test = df_test[target]

# Predictions
y_pred = pipeline.predict(X_test)
y_proba = pipeline.predict_proba(X_test)[:, 1]

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_proba)
conf_matrix = confusion_matrix(y_test, y_pred)

# Print evaluation metrics
print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1-Score: {f1:.2f}')
print(f'ROC-AUC: {roc_auc:.2f}')
print(f'Confusion Matrix:\n{conf_matrix}')

#####FAIRNESS#######
import pandas as pd
from fairlearn.metrics import (
    demographic_parity_ratio, equalized_odds_ratio,
    true_positive_rate_difference, false_positive_rate_difference,
    selection_rate_difference
)

# Function to calculate fairness metrics for a given feature
def calculate_fairness_metrics(y_test, y_pred, X_test, feature):
    metrics = {
        'Feature': feature,
        'Disparate Impact Ratio': demographic_parity_ratio(y_true=y_test, y_pred=y_pred, sensitive_features=X_test[feature]),
        'Equalized Odds Ratio': equalized_odds_ratio(y_true=y_test, y_pred=y_pred, sensitive_features=X_test[feature]),
        'TPR Difference': true_positive_rate_difference(y_true=y_test, y_pred=y_pred, sensitive_features=X_test[feature]),
        'FPR Difference': false_positive_rate_difference(y_true=y_test, y_pred=y_pred, sensitive_features=X_test[feature]),
        'Selection Rate Difference': selection_rate_difference(y_true=y_test, y_pred=y_pred, sensitive_features=X_test[feature])
    }
    return metrics

# Function to calculate disparate impact ratios by group for each sensitive feature
def calculate_disparate_impact(y_true, y_pred, X_test, column):
    disparate_impact_ratios = {}
    overall_favorable_outcomes = y_pred.mean()
    for value in X_test[column].unique():
        mask = X_test[column] == value
        favorable_outcomes = y_pred[mask].mean()
        if overall_favorable_outcomes == 0:
            disparate_impact_ratios[value] = np.nan
        else:
            disparate_impact_ratios[value] = favorable_outcomes / overall_favorable_outcomes
    return disparate_impact_ratios

# Create lists to store fairness metrics and disparate impact ratios
fairness_metrics_list = []
disparate_impact_list = []

# Calculate fairness metrics for each feature
features_to_check = ['sex', 'race', 'age_group', 'priors_count_bin']
for feature in features_to_check:
    metrics = calculate_fairness_metrics(y_test, y_pred, X_test, feature)
    fairness_metrics_list.append(metrics)

    # Calculate and store disparate impact ratios by group
    disparate_impact_by_group = calculate_disparate_impact(y_test, y_pred, X_test, feature)
    for group, ratio in disparate_impact_by_group.items():
        disparate_impact_list.append({'Feature': feature, 'Group': group, 'Disparate Impact Ratio': ratio})

# Convert lists to DataFrames
fairness_metrics_df = pd.DataFrame(fairness_metrics_list)
disparate_impact_df = pd.DataFrame(disparate_impact_list)

# Reorder columns to have 'Feature' as the first column
fairness_metrics_df = fairness_metrics_df[['Feature', 'Disparate Impact Ratio', 'Equalized Odds Ratio', 'TPR Difference', 'FPR Difference', 'Selection Rate Difference']]
disparate_impact_df = disparate_impact_df[['Feature', 'Group', 'Disparate Impact Ratio']]

# Format and print the DataFrames
print("Fairness Metrics:")
print(fairness_metrics_df.round(2).to_string(index=False))

print("\nDisparate Impact Ratios by Group:")
print(disparate_impact_df.round(2).to_string(index=False))


Accuracy: 0.66
Precision: 0.65
Recall: 0.51
F1-Score: 0.57
ROC-AUC: 0.73
Confusion Matrix:
[[500 144]
 [257 270]]


KeyError: 'sex'

In [120]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
)
from fairlearn.preprocessing import CorrelationRemover
from fairlearn.metrics import (
    demographic_parity_ratio, equalized_odds_ratio,
    true_positive_rate_difference, false_positive_rate_difference,
    selection_rate_difference
)
import joblib

# Define preprocess_data function to include binning for total_adult_crimes
def preprocess_data(df):
    df['dob'] = pd.to_datetime(df['dob'])
    df['c_offense_date'] = pd.to_datetime(df['c_offense_date'])
    df['c_arrest_date'] = pd.to_datetime(df['c_arrest_date'])
    df['c_jail_in'] = pd.to_datetime(df['c_jail_in'])

    df['unified_date'] = df['c_arrest_date'].combine_first(df['c_offense_date'])

    df['age_birth'] = (pd.to_datetime('today') - df['dob']).dt.days // 365
    df['age_at_unified_date'] = (df['unified_date'] - df['dob']).dt.days // 365
    
    avg_time_offense_arrest = (df['c_arrest_date'] - df['c_offense_date']).dt.days.mean()
    avg_time_since_jail_in = (pd.to_datetime('today') - df['c_jail_in']).dt.days.mean()
    avg_time_to_jail = (df['c_jail_in'] - df['unified_date']).dt.days.mean()

    df['time_offense_arrest'] = (df['c_arrest_date'] - df['c_offense_date']).dt.days.fillna(avg_time_offense_arrest)
    df['time_since_jail_in'] = (pd.to_datetime('today') - df['c_jail_in']).dt.days.fillna(avg_time_since_jail_in)
    df['time_to_jail'] = (df['c_jail_in'] - df['unified_date']).dt.days.fillna(avg_time_to_jail)
    
    df['total_juv_crimes'] = df['juv_fel_count'] + df['juv_misd_count'] + df['juv_other_count']
    df['total_adult_crimes'] = df['priors_count'] - df['total_juv_crimes']
    
    # Apply frequency encoding
    df['c_charge_desc_freq'] = df['c_charge_desc'].map(df['c_charge_desc'].value_counts(normalize=True))

    # Create bins for total_adult_crimes
    bins = [-np.inf, 0, 1, 3, 5, 10, np.inf]
    labels = ['<0', '0', '1-2', '3-4', '5-9', '10+']
    df['total_adult_crimes_bin'] = pd.cut(df['total_adult_crimes'], bins=bins, labels=labels)
    
    return df

# Assuming df is already loaded and preprocessed
# Apply preprocessing to the entire dataset
df = preprocess_data(df)

# Split data into train and test sets
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

# Define features
all_features = [
    "sex", 
    "race", 
    "total_adult_crimes_bin", 
    "c_charge_degree", 
    "c_charge_desc_freq",
    "age_at_unified_date", 
    "total_juv_crimes",
    "offense_month", 
    "offense_day_of_week", 
    "arrest_month", 
    "arrest_day_of_week"
]

numerical_features = [
    "total_juv_crimes",
    "offense_month", 
    "offense_day_of_week", 
    "arrest_month", 
    "arrest_day_of_week",
    "age_at_unified_date", 
    "c_charge_desc_freq"
]

categorical_features = [
    "sex", 
    "race", 
    "total_adult_crimes_bin", 
    "c_charge_degree"
]

target = 'is_recid'

# Define preprocessing pipelines
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing_value')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Fair representation learning
fair_representation = CorrelationRemover(sensitive_feature_ids=[0, 1])  # Indices of sensitive features (sex and race)

# Use the best model parameters found earlier
best_model_params = {
    'learning_rate': 0.01,
    'max_depth': 3,
    'n_estimators': 300,
    'subsample': 0.8,
    'random_state': 42
}

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('fair_representation', fair_representation),
    ('classifier', GradientBoostingClassifier(**best_model_params))
])

# Fit the model
X_train = df_train[all_features]
y_train = df_train[target]
pipeline.fit(X_train, y_train)

# Evaluate the updated pipeline on the test set
X_test = df_test[all_features]
y_test = df_test[target]

# Predictions
y_pred = pipeline.predict(X_test)
y_proba = pipeline.predict_proba(X_test)[:, 1]

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_proba)
conf_matrix = confusion_matrix(y_test, y_pred)

# Print evaluation metrics
print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1-Score: {f1:.2f}')
print(f'ROC-AUC: {roc_auc:.2f}')
print(f'Confusion Matrix:\n{conf_matrix}')

# Fairness metrics
for feature in ['race']:
    print(f"\nFairness metrics for {feature}:")
    disparate_impact = demographic_parity_ratio(y_true=y_test, y_pred=y_pred, sensitive_features=X_test[feature])
    equalized_odds = equalized_odds_ratio(y_true=y_test, y_pred=y_pred, sensitive_features=X_test[feature])
    tpr_diff = true_positive_rate_difference(y_true=y_test, y_pred=y_pred, sensitive_features=X_test[feature])
    fpr_diff = false_positive_rate_difference(y_true=y_test, y_pred=y_pred, sensitive_features=X_test[feature])
    selection_rate_diff = selection_rate_difference(y_true=y_test, y_pred=y_pred, sensitive_features=X_test[feature])
    print(f"Disparate Impact Ratio: {disparate_impact:.2f}")
    print(f"Equalized Odds Ratio: {equalized_odds:.2f}")
    print(f"True Positive Rate Difference: {tpr_diff:.2f}")
    print(f"False Positive Rate Difference: {fpr_diff:.2f}")
    print(f"Selection Rate Difference: {selection_rate_diff:.2f}")

# Disparate impact ratios by group for each sensitive feature
def calculate_disparate_impact(y_true, y_pred, X_test, column):
    disparate_impact_ratios = {}
    overall_favorable_outcomes = y_pred.mean()
    for value in X_test[column].unique():
        mask = X_test[column] == value
        favorable_outcomes = y_pred[mask].mean()
        if overall_favorable_outcomes == 0:
            disparate_impact_ratios[value] = np.nan
        else:
            disparate_impact_ratios[value] = favorable_outcomes / overall_favorable_outcomes
    return disparate_impact_ratios

for feature in ['sex', 'race', 'total_adult_crimes_bin']:
    disparate_impact_by_group = calculate_disparate_impact(y_test, y_pred, X_test, feature)
    print(f"\nDisparate Impact Ratios by {feature.capitalize()}:")
    for group, ratio in disparate_impact_by_group.items():
        print(f"{group}: {ratio:.2f}")


Accuracy: 0.69
Precision: 0.67
Recall: 0.60
F1-Score: 0.63
ROC-AUC: 0.74
Confusion Matrix:
[[487 157]
 [211 316]]

Fairness metrics for race:
Disparate Impact Ratio: 0.22
Equalized Odds Ratio: 0.00
True Positive Rate Difference: 1.00
False Positive Rate Difference: 0.32
Selection Rate Difference: 0.58

Disparate Impact Ratios by Sex:
Male: 1.11
Female: 0.58

Disparate Impact Ratios by Race:
African-American: 1.26
Asian: 0.41
Caucasian: 0.67
Other: 0.86
Hispanic: 0.86
Native American: 1.86

Disparate Impact Ratios by Total_adult_crimes_bin:
10+: 2.45
3-4: 1.69
<0: 0.45
1-2: 1.27
0: 0.19
5-9: 1.86


In [ ]:
## ANOTHER

In [126]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from imblearn.over_sampling import SMOTE
from fairlearn.metrics import (
    demographic_parity_ratio, equalized_odds_ratio,
    true_positive_rate_difference, false_positive_rate_difference,
    selection_rate_difference
)

# Define preprocess_data function to include binning for total_adult_crimes
def preprocess_data(df):
    df['dob'] = pd.to_datetime(df['dob'])
    df['c_offense_date'] = pd.to_datetime(df['c_offense_date'])
    df['c_arrest_date'] = pd.to_datetime(df['c_arrest_date'])
    df['c_jail_in'] = pd.to_datetime(df['c_jail_in'])

    df['unified_date'] = df['c_arrest_date'].combine_first(df['c_offense_date'])

    df['age_birth'] = (pd.to_datetime('today') - df['dob']).dt.days // 365
    df['age_at_unified_date'] = (df['unified_date'] - df['dob']).dt.days // 365
    
    avg_time_offense_arrest = (df['c_arrest_date'] - df['c_offense_date']).dt.days.mean()
    avg_time_since_jail_in = (pd.to_datetime('today') - df['c_jail_in']).dt.days.mean()
    avg_time_to_jail = (df['c_jail_in'] - df['unified_date']).dt.days.mean()

    df['time_offense_arrest'] = (df['c_arrest_date'] - df['c_offense_date']).dt.days.fillna(avg_time_offense_arrest)
    df['time_since_jail_in'] = (pd.to_datetime('today') - df['c_jail_in']).dt.days.fillna(avg_time_since_jail_in)
    df['time_to_jail'] = (df['c_jail_in'] - df['unified_date']).dt.days.fillna(avg_time_to_jail)
    
    df['total_juv_crimes'] = df['juv_fel_count'] + df['juv_misd_count'] + df['juv_other_count']
    df['total_adult_crimes'] = df['priors_count'] - df['total_juv_crimes']
    
    # Apply frequency encoding
    df['c_charge_desc_freq'] = df['c_charge_desc'].map(df['c_charge_desc'].value_counts(normalize=True))

    # Create bins for total_adult_crimes
    bins = [-np.inf, 0, 1, 3, 5, 10, np.inf]
    labels = ['<0', '0', '1-2', '3-4', '5-9', '10+']
    df['total_adult_crimes_bin'] = pd.cut(df['total_adult_crimes'], bins=bins, labels=labels)

    return df

# Assuming df is already loaded and preprocessed
# Apply preprocessing to the entire dataset
df = preprocess_data(df)

# Split data into train and test sets
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

# Define features
all_features = [
    "sex", 
    "race", 
    "total_adult_crimes_bin", 
    "c_charge_degree", 
    "c_charge_desc_freq",
    "age_at_unified_date", 
    "total_juv_crimes",
    "offense_month", 
    "offense_day_of_week", 
    "arrest_month", 
    "arrest_day_of_week"
]

numerical_features = [
    "total_juv_crimes",
    "offense_month", 
    "offense_day_of_week", 
    "arrest_month", 
    "arrest_day_of_week",
    "age_at_unified_date", 
    "c_charge_desc_freq"
]

categorical_features = [
    "sex", 
    "race", 
    "total_adult_crimes_bin", 
    "c_charge_degree"
]

target = 'is_recid'

# Define preprocessing pipelines
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing_value')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Use the best model parameters found earlier
best_model_params = {
    'learning_rate': 0.01,
    'max_depth': 3,
    'n_estimators': 300,
    'subsample': 0.8,
    'random_state': 42
}

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier(**best_model_params))
])

# Fit the model without sample weights first to get preprocessed data for SMOTE
X_train = df_train[all_features]
y_train = df_train[target]

# Apply preprocessing to the training data before SMOTE
X_train_transformed = preprocessor.fit_transform(X_train)

# Apply SMOTE to address class imbalance
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_transformed, y_train)

# Fit the model using resampled data
pipeline.named_steps['classifier'].fit(X_train_resampled, y_train_resampled)

# Transform the test data
X_test_transformed = preprocessor.transform(df_test[all_features])
y_test = df_test[target]

# Predictions
y_pred = pipeline.named_steps['classifier'].predict(X_test_transformed)
y_proba = pipeline.named_steps['classifier'].predict_proba(X_test_transformed)[:, 1]

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_proba)
conf_matrix = confusion_matrix(y_test, y_pred)

# Print evaluation metrics
print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1-Score: {f1:.2f}')
print(f'ROC-AUC: {roc_auc:.2f}')
print(f'Confusion Matrix:\n{conf_matrix}')

# Fairness metrics
for feature in ['sex', 'race', 'total_adult_crimes_bin']:
    print(f"\nFairness metrics for {feature}:")
    disparate_impact = demographic_parity_ratio(y_true=y_test, y_pred=y_pred, sensitive_features=X_test[feature])
    equalized_odds = equalized_odds_ratio(y_true=y_test, y_pred=y_pred, sensitive_features=X_test[feature])
    tpr_diff = true_positive_rate_difference(y_true=y_test, y_pred=y_pred, sensitive_features=X_test[feature])
    fpr_diff = false_positive_rate_difference(y_true=y_test, y_pred=y_pred, sensitive_features=X_test[feature])
    selection_rate_diff = selection_rate_difference(y_true=y_test, y_pred=y_pred, sensitive_features=X_test[feature])
    print(f"Disparate Impact Ratio: {disparate_impact:.2f}")
    print(f"Equalized Odds Ratio: {equalized_odds:.2f}")
    print(f"True Positive Rate Difference: {tpr_diff:.2f}")
    print(f"False Positive Rate Difference: {fpr_diff:.2f}")
    print(f"Selection Rate Difference: {selection_rate_diff:.2f}")

# Disparate impact ratios by group for each sensitive feature
def calculate_disparate_impact(y_true, y_pred, X_test, column):
    disparate_impact_ratios = {}
    overall_favorable_outcomes = y_pred.mean()
    for value in X_test[column].unique():
        mask = X_test[column] == value
        favorable_outcomes = y_pred[mask].mean()
        if overall_favorable_outcomes == 0:
            disparate_impact_ratios[value] = np.nan
        else:
            disparate_impact_ratios[value] = favorable_outcomes / overall_favorable_outcomes
    return disparate_impact_ratios

for feature in ['sex', 'race', 'total_adult_crimes_bin']:
    disparate_impact_by_group = calculate_disparate_impact(y_test, y_pred, X_test, feature)
    print(f"\nDisparate Impact Ratios by {feature.capitalize()}:")
    for group, ratio in disparate_impact_by_group.items():
        print(f"{group}: {ratio:.2f}")


Accuracy: 0.67
Precision: 0.62
Recall: 0.68
F1-Score: 0.65
ROC-AUC: 0.74
Confusion Matrix:
[[424 220]
 [167 360]]

Fairness metrics for sex:
Disparate Impact Ratio: 0.59
Equalized Odds Ratio: 0.52
True Positive Rate Difference: 0.17
False Positive Rate Difference: 0.18
Selection Rate Difference: 0.22

Fairness metrics for race:
Disparate Impact Ratio: 0.22
Equalized Odds Ratio: 0.00
True Positive Rate Difference: 1.00
False Positive Rate Difference: 0.48
Selection Rate Difference: 0.58

Fairness metrics for total_adult_crimes_bin:
Disparate Impact Ratio: 0.23
Equalized Odds Ratio: 0.12
True Positive Rate Difference: 0.66
False Positive Rate Difference: 0.88
Selection Rate Difference: 0.77

Disparate Impact Ratios by Sex:
Male: 1.09
Female: 0.65

Disparate Impact Ratios by Race:
African-American: 1.28
Asian: 0.34
Caucasian: 0.69
Other: 0.70
Hispanic: 0.76
Native American: 1.51

Disparate Impact Ratios by Total_adult_crimes_bin:
10+: 2.02
3-4: 1.44
<0: 0.46
1-2: 1.24
0: 0.50
5-9: 1.88


In [ ]:
##ANOTHER TRIAL:

In [127]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, VotingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from imblearn.over_sampling import SMOTE
from fairlearn.metrics import (
    demographic_parity_ratio, equalized_odds_ratio,
    true_positive_rate_difference, false_positive_rate_difference,
    selection_rate_difference
)

# Define preprocess_data function to include binning for total_adult_crimes
def preprocess_data(df):
    df['dob'] = pd.to_datetime(df['dob'])
    df['c_offense_date'] = pd.to_datetime(df['c_offense_date'])
    df['c_arrest_date'] = pd.to_datetime(df['c_arrest_date'])
    df['c_jail_in'] = pd.to_datetime(df['c_jail_in'])

    df['unified_date'] = df['c_arrest_date'].combine_first(df['c_offense_date'])

    df['age_birth'] = (pd.to_datetime('today') - df['dob']).dt.days // 365
    df['age_at_unified_date'] = (df['unified_date'] - df['dob']).dt.days // 365
    
    avg_time_offense_arrest = (df['c_arrest_date'] - df['c_offense_date']).dt.days.mean()
    avg_time_since_jail_in = (pd.to_datetime('today') - df['c_jail_in']).dt.days.mean()
    avg_time_to_jail = (df['c_jail_in'] - df['unified_date']).dt.days.mean()

    df['time_offense_arrest'] = (df['c_arrest_date'] - df['c_offense_date']).dt.days.fillna(avg_time_offense_arrest)
    df['time_since_jail_in'] = (pd.to_datetime('today') - df['c_jail_in']).dt.days.fillna(avg_time_since_jail_in)
    df['time_to_jail'] = (df['c_jail_in'] - df['unified_date']).dt.days.fillna(avg_time_to_jail)
    
    df['total_juv_crimes'] = df['juv_fel_count'] + df['juv_misd_count'] + df['juv_other_count']
    df['total_adult_crimes'] = df['priors_count'] - df['total_juv_crimes']
    
    # Apply frequency encoding
    df['c_charge_desc_freq'] = df['c_charge_desc'].map(df['c_charge_desc'].value_counts(normalize=True))

    # Create bins for total_adult_crimes
    bins = [-np.inf, 0, 1, 3, 5, 10, np.inf]
    labels = ['<0', '0', '1-2', '3-4', '5-9', '10+']
    df['total_adult_crimes_bin'] = pd.cut(df['total_adult_crimes'], bins=bins, labels=labels)

    return df

# Assuming df is already loaded and preprocessed
# Apply preprocessing to the entire dataset
df = preprocess_data(df)

# Split data into train and test sets
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

# Define features
all_features = [
    "sex", 
    "race", 
    "total_adult_crimes_bin", 
    "c_charge_degree", 
    "c_charge_desc_freq",
    "age_at_unified_date", 
    "total_juv_crimes",
    "offense_month", 
    "offense_day_of_week", 
    "arrest_month", 
    "arrest_day_of_week"
]

numerical_features = [
    "total_juv_crimes",
    "offense_month", 
    "offense_day_of_week", 
    "arrest_month", 
    "arrest_day_of_week",
    "age_at_unified_date", 
    "c_charge_desc_freq"
]

categorical_features = [
    "sex", 
    "race", 
    "total_adult_crimes_bin", 
    "c_charge_degree"
]

target = 'is_recid'

# Define preprocessing pipelines
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures(degree=2, include_bias=False))
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing_value')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Define ensemble model
rf_model = RandomForestClassifier(random_state=42)
gb_model = GradientBoostingClassifier(random_state=42)
voting_model = VotingClassifier(estimators=[('rf', rf_model), ('gb', gb_model)], voting='soft')

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', voting_model)
])

# Apply preprocessing to the training data before SMOTE
X_train = df_train[all_features]
y_train = df_train[target]
X_train_transformed = preprocessor.fit_transform(X_train)

# Apply SMOTE to address class imbalance
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_transformed, y_train)

# Fit the model using resampled data
pipeline.named_steps['classifier'].fit(X_train_resampled, y_train_resampled)

# Transform the test data
X_test_transformed = preprocessor.transform(df_test[all_features])
y_test = df_test[target]

# Predictions
y_pred = pipeline.named_steps['classifier'].predict(X_test_transformed)
y_proba = pipeline.named_steps['classifier'].predict_proba(X_test_transformed)[:, 1]

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_proba)
conf_matrix = confusion_matrix(y_test, y_pred)

# Print evaluation metrics
print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1-Score: {f1:.2f}')
print(f'ROC-AUC: {roc_auc:.2f}')
print(f'Confusion Matrix:\n{conf_matrix}')

# Fairness metrics
for feature in ['sex', 'race', 'total_adult_crimes_bin']:
    print(f"\nFairness metrics for {feature}:")
    disparate_impact = demographic_parity_ratio(y_true=y_test, y_pred=y_pred, sensitive_features=X_test[feature])
    equalized_odds = equalized_odds_ratio(y_true=y_test, y_pred=y_pred, sensitive_features=X_test[feature])
    tpr_diff = true_positive_rate_difference(y_true=y_test, y_pred=y_pred, sensitive_features=X_test[feature])
    fpr_diff = false_positive_rate_difference(y_true=y_test, y_pred=y_pred, sensitive_features=X_test[feature])
    selection_rate_diff = selection_rate_difference(y_true=y_test, y_pred=y_pred, sensitive_features=X_test[feature])
    print(f"Disparate Impact Ratio: {disparate_impact:.2f}")
    print(f"Equalized Odds Ratio: {equalized_odds:.2f}")
    print(f"True Positive Rate Difference: {tpr_diff:.2f}")
    print(f"False Positive Rate Difference: {fpr_diff:.2f}")
    print(f"Selection Rate Difference: {selection_rate_diff:.2f}")

# Disparate impact ratios by group for each sensitive feature
def calculate_disparate_impact(y_true, y_pred, X_test, column):
    disparate_impact_ratios = {}
    overall_favorable_outcomes = y_pred.mean()
    for value in X_test[column].unique():
        mask = X_test[column] == value
        favorable_outcomes = y_pred[mask].mean()
        if overall_favorable_outcomes == 0:
            disparate_impact_ratios[value] = np.nan
        else:
            disparate_impact_ratios[value] = favorable_outcomes / overall_favorable_outcomes
    return disparate_impact_ratios

for feature in ['sex', 'race', 'total_adult_crimes_bin']:
    disparate_impact_by_group = calculate_disparate_impact(y_test, y_pred, X_test, feature)
    print(f"\nDisparate Impact Ratios by {feature.capitalize()}:")
    for group, ratio in disparate_impact_by_group.items():
        print(f"{group}: {ratio:.2f}")


Accuracy: 0.66
Precision: 0.63
Recall: 0.62
F1-Score: 0.62
ROC-AUC: 0.73
Confusion Matrix:
[[449 195]
 [201 326]]

Fairness metrics for sex:
Disparate Impact Ratio: 0.61
Equalized Odds Ratio: 0.52
True Positive Rate Difference: 0.12
False Positive Rate Difference: 0.17
Selection Rate Difference: 0.19

Fairness metrics for race:
Disparate Impact Ratio: 0.41
Equalized Odds Ratio: 0.00
True Positive Rate Difference: 1.00
False Positive Rate Difference: 0.60
Selection Rate Difference: 0.44

Fairness metrics for total_adult_crimes_bin:
Disparate Impact Ratio: 0.22
Equalized Odds Ratio: 0.12
True Positive Rate Difference: 0.54
False Positive Rate Difference: 0.88
Selection Rate Difference: 0.73

Disparate Impact Ratios by Sex:
Male: 1.09
Female: 0.67

Disparate Impact Ratios by Race:
African-American: 1.24
Asian: 1.12
Caucasian: 0.72
Other: 0.69
Hispanic: 0.82
Native American: 1.69

Disparate Impact Ratios by Total_adult_crimes_bin:
10+: 2.10
3-4: 1.55
<0: 0.46
1-2: 1.21
0: 0.55
5-9: 1.72


In [ ]:
##OTHER

In [128]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from imblearn.over_sampling import SMOTE
from fairlearn.metrics import (
    demographic_parity_ratio, equalized_odds_ratio,
    true_positive_rate_difference, false_positive_rate_difference,
    selection_rate_difference
)

# Define preprocess_data function to include binning for total_adult_crimes
def preprocess_data(df):
    df['dob'] = pd.to_datetime(df['dob'])
    df['c_offense_date'] = pd.to_datetime(df['c_offense_date'])
    df['c_arrest_date'] = pd.to_datetime(df['c_arrest_date'])
    df['c_jail_in'] = pd.to_datetime(df['c_jail_in'])

    df['unified_date'] = df['c_arrest_date'].combine_first(df['c_offense_date'])

    df['age_birth'] = (pd.to_datetime('today') - df['dob']).dt.days // 365
    df['age_at_unified_date'] = (df['unified_date'] - df['dob']).dt.days // 365
    
    avg_time_offense_arrest = (df['c_arrest_date'] - df['c_offense_date']).dt.days.mean()
    avg_time_since_jail_in = (pd.to_datetime('today') - df['c_jail_in']).dt.days.mean()
    avg_time_to_jail = (df['c_jail_in'] - df['unified_date']).dt.days.mean()

    df['time_offense_arrest'] = (df['c_arrest_date'] - df['c_offense_date']).dt.days.fillna(avg_time_offense_arrest)
    df['time_since_jail_in'] = (pd.to_datetime('today') - df['c_jail_in']).dt.days.fillna(avg_time_since_jail_in)
    df['time_to_jail'] = (df['c_jail_in'] - df['unified_date']).dt.days.fillna(avg_time_to_jail)
    
    df['total_juv_crimes'] = df['juv_fel_count'] + df['juv_misd_count'] + df['juv_other_count']
    df['total_adult_crimes'] = df['priors_count'] - df['total_juv_crimes']
    
    # Apply frequency encoding
    df['c_charge_desc_freq'] = df['c_charge_desc'].map(df['c_charge_desc'].value_counts(normalize=True))

    # Create bins for total_adult_crimes
    bins = [-np.inf, 0, 1, 3, 5, 10, np.inf]
    labels = ['<0', '0', '1-2', '3-4', '5-9', '10+']
    df['total_adult_crimes_bin'] = pd.cut(df['total_adult_crimes'], bins=bins, labels=labels)
    
    return df

# Assuming df is already loaded and preprocessed
# Apply preprocessing to the entire dataset
df = preprocess_data(df)

# Split data into train and test sets
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

# Define features
all_features = [
    "sex", 
    "race", 
    "total_adult_crimes_bin", 
    "c_charge_degree", 
    "c_charge_desc_freq",
    "age_at_unified_date", 
    "total_juv_crimes",
    "offense_month", 
    "offense_day_of_week", 
    "arrest_month", 
    "arrest_day_of_week"
]

numerical_features = [
    "total_juv_crimes",
    "offense_month", 
    "offense_day_of_week", 
    "arrest_month", 
    "arrest_day_of_week",
    "age_at_unified_date", 
    "c_charge_desc_freq"
]

categorical_features = [
    "sex", 
    "race", 
    "total_adult_crimes_bin", 
    "c_charge_degree"
]

target = 'is_recid'

# Define preprocessing pipelines
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing_value')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Define base models for stacking
base_models = [
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('gb', GradientBoostingClassifier(n_estimators=100, random_state=42))
]

# Define stacking ensemble
stacking_model = StackingClassifier(
    estimators=base_models,
    final_estimator=LogisticRegression(),
    cv=5
)

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', stacking_model)
])

# Apply preprocessing to the training data before SMOTE
X_train = df_train[all_features]
y_train = df_train[target]
X_train_transformed = preprocessor.fit_transform(X_train)

# Apply SMOTE to address class imbalance
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_transformed, y_train)

# Fit the model using resampled data
pipeline.named_steps['classifier'].fit(X_train_resampled, y_train_resampled)

# Transform the test data
X_test_transformed = preprocessor.transform(df_test[all_features])
y_test = df_test[target]

# Predictions
y_pred = pipeline.named_steps['classifier'].predict(X_test_transformed)
y_proba = pipeline.named_steps['classifier'].predict_proba(X_test_transformed)[:, 1]

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_proba)
conf_matrix = confusion_matrix(y_test, y_pred)

# Print evaluation metrics
print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1-Score: {f1:.2f}')
print(f'ROC-AUC: {roc_auc:.2f}')
print(f'Confusion Matrix:\n{conf_matrix}')

# Fairness metrics
for feature in ['sex', 'race', 'total_adult_crimes_bin']:
    print(f"\nFairness metrics for {feature}:")
    disparate_impact = demographic_parity_ratio(y_true=y_test, y_pred=y_pred, sensitive_features=X_test[feature])
    equalized_odds = equalized_odds_ratio(y_true=y_test, y_pred=y_pred, sensitive_features=X_test[feature])
    tpr_diff = true_positive_rate_difference(y_true=y_test, y_pred=y_pred, sensitive_features=X_test[feature])
    fpr_diff = false_positive_rate_difference(y_true=y_test, y_pred=y_pred, sensitive_features=X_test[feature])
    selection_rate_diff = selection_rate_difference(y_true=y_test, y_pred=y_pred, sensitive_features=X_test[feature])
    print(f"Disparate Impact Ratio: {disparate_impact:.2f}")
    print(f"Equalized Odds Ratio: {equalized_odds:.2f}")
    print(f"True Positive Rate Difference: {tpr_diff:.2f}")
    print(f"False Positive Rate Difference: {fpr_diff:.2f}")
    print(f"Selection Rate Difference: {selection_rate_diff:.2f}")

# Disparate impact ratios by group for each sensitive feature
def calculate_disparate_impact(y_true, y_pred, X_test, column):
    disparate_impact_ratios = {}
    overall_favorable_outcomes = y_pred.mean()
    for value in X_test[column].unique():
        mask = X_test[column] == value
        favorable_outcomes = y_pred[mask].mean()
        if overall_favorable_outcomes == 0:
            disparate_impact_ratios[value] = np.nan
        else:
            disparate_impact_ratios[value] = favorable_outcomes / overall_favorable_outcomes
    return disparate_impact_ratios

for feature in ['sex', 'race', 'total_adult_crimes_bin']:
    disparate_impact_by_group = calculate_disparate_impact(y_test, y_pred, X_test, feature)
    print(f"\nDisparate Impact Ratios by {feature.capitalize()}:")
    for group, ratio in disparate_impact_by_group.items():
        print(f"{group}: {ratio:.2f}")


Accuracy: 0.67
Precision: 0.64
Recall: 0.65
F1-Score: 0.64
ROC-AUC: 0.73
Confusion Matrix:
[[447 197]
 [184 343]]

Fairness metrics for sex:
Disparate Impact Ratio: 0.47
Equalized Odds Ratio: 0.33
True Positive Rate Difference: 0.21
False Positive Rate Difference: 0.25
Selection Rate Difference: 0.28

Fairness metrics for race:
Disparate Impact Ratio: 0.56
Equalized Odds Ratio: 0.00
True Positive Rate Difference: 0.75
False Positive Rate Difference: 0.41
Selection Rate Difference: 0.25

Fairness metrics for total_adult_crimes_bin:
Disparate Impact Ratio: 0.22
Equalized Odds Ratio: 0.12
True Positive Rate Difference: 0.52
False Positive Rate Difference: 0.85
Selection Rate Difference: 0.73

Disparate Impact Ratios by Sex:
Male: 1.12
Female: 0.53

Disparate Impact Ratios by Race:
African-American: 1.26
Asian: 0.72
Caucasian: 0.72
Other: 0.71
Hispanic: 0.77
Native American: 1.08

Disparate Impact Ratios by Total_adult_crimes_bin:
10+: 2.02
3-4: 1.55
<0: 0.45
1-2: 1.21
0: 0.59
5-9: 1.74


In [131]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from xgboost import XGBClassifier
from skopt import BayesSearchCV
from imblearn.over_sampling import SMOTE
from fairlearn.metrics import (
    demographic_parity_ratio, equalized_odds_ratio,
    true_positive_rate_difference, false_positive_rate_difference,
    selection_rate_difference
)

# Define preprocess_data function to include binning for total_adult_crimes
def preprocess_data(df):
    df['dob'] = pd.to_datetime(df['dob'])
    df['c_offense_date'] = pd.to_datetime(df['c_offense_date'])
    df['c_arrest_date'] = pd.to_datetime(df['c_arrest_date'])
    df['c_jail_in'] = pd.to_datetime(df['c_jail_in'])

    df['unified_date'] = df['c_arrest_date'].combine_first(df['c_offense_date'])

    df['age_birth'] = (pd.to_datetime('today') - df['dob']).dt.days // 365
    df['age_at_unified_date'] = (df['unified_date'] - df['dob']).dt.days // 365
    
    avg_time_offense_arrest = (df['c_arrest_date'] - df['c_offense_date']).dt.days.mean()
    avg_time_since_jail_in = (pd.to_datetime('today') - df['c_jail_in']).dt.days.mean()
    avg_time_to_jail = (df['c_jail_in'] - df['unified_date']).dt.days.mean()

    df['time_offense_arrest'] = (df['c_arrest_date'] - df['c_offense_date']).dt.days.fillna(avg_time_offense_arrest)
    df['time_since_jail_in'] = (pd.to_datetime('today') - df['c_jail_in']).dt.days.fillna(avg_time_since_jail_in)
    df['time_to_jail'] = (df['c_jail_in'] - df['unified_date']).dt.days.fillna(avg_time_to_jail)
    
    df['total_juv_crimes'] = df['juv_fel_count'] + df['juv_misd_count'] + df['juv_other_count']
    df['total_adult_crimes'] = df['priors_count'] - df['total_juv_crimes']
    
    # Apply frequency encoding
    df['c_charge_desc_freq'] = df['c_charge_desc'].map(df['c_charge_desc'].value_counts(normalize=True))

    # Create bins for total_adult_crimes
    bins = [-np.inf, 0, 1, 3, 5, 10, np.inf]
    labels = ['<0', '0', '1-2', '3-4', '5-9', '10+']
    df['total_adult_crimes_bin'] = pd.cut(df['total_adult_crimes'], bins=bins, labels=labels)
    
    return df

# Assuming df is already loaded and preprocessed
# Apply preprocessing to the entire dataset
df = preprocess_data(df)

# Split data into train and test sets
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

# Define features
all_features = [
    "sex", 
    "race", 
    "total_adult_crimes_bin", 
    "c_charge_degree", 
    "c_charge_desc_freq",
    "age_at_unified_date", 
    "total_juv_crimes",
    "offense_month", 
    "offense_day_of_week", 
    "arrest_month", 
    "arrest_day_of_week"
]

numerical_features = [
    "total_juv_crimes",
    "offense_month", 
    "offense_day_of_week", 
    "arrest_month", 
    "arrest_day_of_week",
    "age_at_unified_date", 
    "c_charge_desc_freq"
]

categorical_features = [
    "sex", 
    "race", 
    "total_adult_crimes_bin", 
    "c_charge_degree"
]

target = 'is_recid'

# Define preprocessing pipelines
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing_value')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Define XGBoost model and parameter search space for Bayesian optimization
xgb = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')

param_grid = {
    'learning_rate': (0.01, 1.0, 'log-uniform'),
    'max_depth': (3, 10),
    'n_estimators': (100, 1000),
    'subsample': (0.5, 1.0),
    'colsample_bytree': (0.5, 1.0)
}

bayes_cv = BayesSearchCV(
    estimator=xgb,
    search_spaces=param_grid,
    n_iter=32,
    cv=5,
    random_state=42,
    scoring='roc_auc',
    n_jobs=-1
)

# Apply preprocessing to the training data before SMOTE
X_train = df_train[all_features]
y_train = df_train[target]
X_train_transformed = preprocessor.fit_transform(X_train)

# Apply SMOTE to address class imbalance
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_transformed, y_train)

# Fit the model using resampled data
bayes_cv.fit(X_train_resampled, y_train_resampled)

# Transform the test data
X_test_transformed = preprocessor.transform(df_test[all_features])
y_test = df_test[target]

# Predictions
y_pred = bayes_cv.predict(X_test_transformed)
y_proba = bayes_cv.predict_proba(X_test_transformed)[:, 1]

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_proba)
conf_matrix = confusion_matrix(y_test, y_pred)

# Print evaluation metrics
print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1-Score: {f1:.2f}')
print(f'ROC-AUC: {roc_auc:.2f}')
print(f'Confusion Matrix:\n{conf_matrix}')

# Fairness metrics
for feature in ['sex', 'race', 'total_adult_crimes_bin']:
    print(f"\nFairness metrics for {feature}:")
    disparate_impact = demographic_parity_ratio(y_true=y_test, y_pred=y_pred, sensitive_features=df_test[feature])
    equalized_odds = equalized_odds_ratio(y_true=y_test, y_pred=y_pred, sensitive_features=df_test[feature])
    tpr_diff = true_positive_rate_difference(y_true=y_test, y_pred=y_pred, sensitive_features=df_test[feature])
    fpr_diff = false_positive_rate_difference(y_true=y_test, y_pred=y_pred, sensitive_features=df_test[feature])
    selection_rate_diff = selection_rate_difference(y_true=y_test, y_pred=y_pred, sensitive_features=df_test[feature])
    print(f"Disparate Impact Ratio: {disparate_impact:.2f}")
    print(f"Equalized Odds Ratio: {equalized_odds:.2f}")
    print(f"True Positive Rate Difference: {tpr_diff:.2f}")
    print(f"False Positive Rate Difference: {fpr_diff:.2f}")
    print(f"Selection Rate Difference: {selection_rate_diff:.2f}")

# Disparate impact ratios by group for each sensitive feature
def calculate_disparate_impact(y_true, y_pred, X_test, column):
    disparate_impact_ratios = {}
    overall_favorable_outcomes = y_pred.mean()
    for value in X_test[column].unique():
        mask = X_test[column] == value
        favorable_outcomes = y_pred[mask].mean()
        if overall_favorable_outcomes == 0:
            disparate_impact_ratios[value] = np.nan
        else:
            disparate_impact_ratios[value] = favorable_outcomes / overall_favorable_outcomes
    return disparate_impact_ratios

for feature in ['sex', 'race', 'total_adult_crimes_bin']:
    disparate_impact_by_group = calculate_disparate_impact(y_test, y_pred, df_test, feature)
    print(f"\nDisparate Impact Ratios by {feature.capitalize()}:")
    for group, ratio in disparate_impact_by_group.items():
        print(f"{group}: {ratio:.2f}")


/home/teresaramoos/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [21:28:44] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/teresaramoos/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [21:28:44] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/teresaramoos/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [21:28:44] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/teresaramoos/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [21:28:44] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/teresaramoos/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [21

/home/teresaramoos/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [21:29:06] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/teresaramoos/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [21:29:06] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/teresaramoos/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [21:29:06] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/teresaramoos/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [21:29:06] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/teresaramoos/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [21

/home/teresaramoos/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [21:29:31] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/teresaramoos/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [21:29:31] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/teresaramoos/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [21:29:31] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/teresaramoos/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [21:29:31] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/teresaramoos/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [21

/home/teresaramoos/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [21:30:14] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/teresaramoos/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [21:30:14] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/teresaramoos/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [21:30:14] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/teresaramoos/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [21:30:14] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/teresaramoos/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [21

/home/teresaramoos/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [21:30:38] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy: 0.67
Precision: 0.63
Recall: 0.65
F1-Score: 0.64
ROC-AUC: 0.74
Confusion Matrix:
[[445 199]
 [184 343]]

Fairness metrics for sex:
Disparate Impact Ratio: 0.48
Equalized Odds Ratio: 0.38
True Positive Rate Difference: 0.24
False Positive Rate Difference: 0.22
Selection Rate Difference: 0.27

Fairness metrics for race:
Disparate Impact Ratio: 0.22
Equalized Odds Ratio: 0.00
True Positive Rate Difference: 1.00
False Positive Rate Difference: 0.42
Selection Rate Difference: 0.58

Fairness metrics for total_adult_crimes_bin:
Disparate Impact Ratio: 0.21
Equalized Odds Ratio: 0.10
True Positive Rate Difference: 0.56
False Positive Rate Difference: 0.86
Selection Rate Difference: 0.75

Disparate Impact Ratios by Sex:
Male: 1.12
Female: 0.53

Disparate Impact Ratios by Race:
African-American: 1.26
Asian: 0.36
Caucasian: 0.74
Other: 0.58
Hispanic: 0.73
Native American: 1.62

Disparate Impact Ratios by Total_adult_crimes_bin:
10+: 2.06
3-4: 1.53
<0: 0.43
1-2: 1.14
0: 0.57
5-9: 1.90


In [133]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

# Preprocess data function
def preprocess_data(df):
    df['dob'] = pd.to_datetime(df['dob'])
    df['c_offense_date'] = pd.to_datetime(df['c_offense_date'])
    df['c_arrest_date'] = pd.to_datetime(df['c_arrest_date'])
    df['c_jail_in'] = pd.to_datetime(df['c_jail_in'])
    df['unified_date'] = df['c_arrest_date'].combine_first(df['c_offense_date'])
    df['age_birth'] = (pd.to_datetime('today') - df['dob']).dt.days // 365
    df['age_at_unified_date'] = (df['unified_date'] - df['dob']).dt.days // 365
    
    avg_time_offense_arrest = (df['c_arrest_date'] - df['c_offense_date']).dt.days.mean()
    avg_time_since_jail_in = (pd.to_datetime('today') - df['c_jail_in']).dt.days.mean()
    avg_time_to_jail = (df['c_jail_in'] - df['unified_date']).dt.days.mean()

    df['time_offense_arrest'] = (df['c_arrest_date'] - df['c_offense_date']).dt.days.fillna(avg_time_offense_arrest)
    df['time_since_jail_in'] = (pd.to_datetime('today') - df['c_jail_in']).dt.days.fillna(avg_time_since_jail_in)
    df['time_to_jail'] = (df['c_jail_in'] - df['unified_date']).dt.days.fillna(avg_time_to_jail)
    
    df['total_juv_crimes'] = df['juv_fel_count'] + df['juv_misd_count'] + df['juv_other_count']
    df['total_adult_crimes'] = df['priors_count'] - df['total_juv_crimes']
    df['c_charge_desc_freq'] = df['c_charge_desc'].map(df['c_charge_desc'].value_counts(normalize=True))

    bins = [-np.inf, 0, 1, 3, 5, 10, np.inf]
    labels = ['<0', '0', '1-2', '3-4', '5-9', '10+']
    df['total_adult_crimes_bin'] = pd.cut(df['total_adult_crimes'], bins=bins, labels=labels)
    
    return df

# Load and preprocess data
df = preprocess_data(df)

# Split data into train and test sets
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

# Define features
all_features = ["sex", "race", "total_adult_crimes_bin", "c_charge_degree", "c_charge_desc_freq", "age_at_unified_date", "total_juv_crimes", "offense_month", "offense_day_of_week", "arrest_month", "arrest_day_of_week"]
numerical_features = ["total_juv_crimes", "offense_month", "offense_day_of_week", "arrest_month", "arrest_day_of_week", "age_at_unified_date", "c_charge_desc_freq"]
categorical_features = ["sex", "race", "total_adult_crimes_bin", "c_charge_degree"]
target = 'is_recid'

# Define preprocessing pipelines
numeric_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='median')), ('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='constant', fill_value='missing_value')), ('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, numerical_features), ('cat', categorical_transformer, categorical_features)])

# Compute sample weights
def compute_sample_weights(df, sensitive_feature):
    group_counts = df[sensitive_feature].value_counts()
    total_count = len(df)
    sample_weights = df[sensitive_feature].apply(lambda x: total_count / (len(group_counts) * group_counts[x]))
    return sample_weights

# Apply reweighing for the 'race' feature
sample_weights = compute_sample_weights(df_train, 'race')

# Define model parameters
best_model_params = {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 300, 'subsample': 0.8, 'random_state': 42}
pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', GradientBoostingClassifier(**best_model_params))])

# Train the model with sample weights
X_train = df_train[all_features]
y_train = df_train[target]
pipeline.fit(X_train, y_train, classifier__sample_weight=sample_weights)

# Evaluate the model
X_test = df_test[all_features]
y_test = df_test[target]
y_pred = pipeline.predict(X_test)
roc_auc = roc_auc_score(y_test, y_pred)

print(f'ROC-AUC: {roc_auc:.2f}')
# Fairness metrics
for feature in ['sex', 'race', 'total_adult_crimes_bin']:
    print(f"\nFairness metrics for {feature}:")
    disparate_impact = demographic_parity_ratio(y_true=y_test, y_pred=y_pred, sensitive_features=df_test[feature])
    equalized_odds = equalized_odds_ratio(y_true=y_test, y_pred=y_pred, sensitive_features=df_test[feature])
    tpr_diff = true_positive_rate_difference(y_true=y_test, y_pred=y_pred, sensitive_features=df_test[feature])
    fpr_diff = false_positive_rate_difference(y_true=y_test, y_pred=y_pred, sensitive_features=df_test[feature])
    selection_rate_diff = selection_rate_difference(y_true=y_test, y_pred=y_pred, sensitive_features=df_test[feature])
    print(f"Disparate Impact Ratio: {disparate_impact:.2f}")
    print(f"Equalized Odds Ratio: {equalized_odds:.2f}")
    print(f"True Positive Rate Difference: {tpr_diff:.2f}")
    print(f"False Positive Rate Difference: {fpr_diff:.2f}")
    print(f"Selection Rate Difference: {selection_rate_diff:.2f}")

# Disparate impact ratios by group for each sensitive feature
def calculate_disparate_impact(y_true, y_pred, X_test, column):
    disparate_impact_ratios = {}
    overall_favorable_outcomes = y_pred.mean()
    for value in X_test[column].unique():
        mask = X_test[column] == value
        favorable_outcomes = y_pred[mask].mean()
        if overall_favorable_outcomes == 0:
            disparate_impact_ratios[value] = np.nan
        else:
            disparate_impact_ratios[value] = favorable_outcomes / overall_favorable_outcomes
    return disparate_impact_ratios

for feature in ['sex', 'race', 'total_adult_crimes_bin']:
    disparate_impact_by_group = calculate_disparate_impact(y_test, y_pred, df_test, feature)
    print(f"\nDisparate Impact Ratios by {feature.capitalize()}:")
    for group, ratio in disparate_impact_by_group.items():
        print(f"{group}: {ratio:.2f}")


ROC-AUC: 0.65

Fairness metrics for sex:
Disparate Impact Ratio: 0.56
Equalized Odds Ratio: 0.56
True Positive Rate Difference: 0.18
False Positive Rate Difference: 0.11
Selection Rate Difference: 0.17

Fairness metrics for race:
Disparate Impact Ratio: 0.33
Equalized Odds Ratio: 0.00
True Positive Rate Difference: 0.67
False Positive Rate Difference: 0.30
Selection Rate Difference: 0.33

Fairness metrics for total_adult_crimes_bin:
Disparate Impact Ratio: 0.09
Equalized Odds Ratio: 0.06
True Positive Rate Difference: 0.81
False Positive Rate Difference: 0.87
Selection Rate Difference: 0.84

Disparate Impact Ratios by Sex:
Male: 1.10
Female: 0.61

Disparate Impact Ratios by Race:
African-American: 1.29
Asian: 0.48
Caucasian: 0.68
Other: 0.72
Hispanic: 0.71
Native American: 1.43

Disparate Impact Ratios by Total_adult_crimes_bin:
10+: 2.64
3-4: 2.47
<0: 0.36
1-2: 0.62
0: 0.23
5-9: 2.29


In [68]:
## Serialization

In [74]:
# Serialization
TMP_DIR = ''

# Serialize the column names from the X DataFrame into a file named columns.json
# YOUR CODE HERE
with open(os.path.join(TMP_DIR, "columns.json"), 'w') as fh:
    json.dump(X_train.columns.tolist(), fh)

# Pickle the dtypes of the columns from the X DataFrame into a file named dtypes.pickle
# YOUR CODE HERE
with open(os.path.join(TMP_DIR,'dtypes.pickle'), 'wb') as fh:
    pickle.dump(X_train.dtypes, fh)
    
# Pickle the fitted pipeline into a file named pipeline.pickle
# YOUR CODE HERE
import joblib
joblib.dump(pipeline, os.path.join(TMP_DIR,'pipeline.pickle'))

# Baseline Scoring
roc_auc_score(y_test, y_pred)
#roc_auc_score(y_test, y_proba)

0.6804188716159676

In [ ]:
## Testing the App

In [33]:
import requests
import json

url = "http://127.0.0.1:5000/will_recidivate/"

data = {
    "id": None,
    "name": "John Doe2",
    "sex": "Male",
    "dob": "1990-01-01",
    "race": "White",
    "juv_fel_count": 0,
    "juv_misd_count": 1,
    "juv_other_count": 0,
    "priors_count": 2,
    "c_case_number": "case1234",
    "c_charge_degree": "M",
    "c_charge_desc": "Theft",
    "c_offense_date": None,
    "c_arrest_date": "2023-01-02",
    "c_jail_in": "2023-01-03"
}


headers = {'Content-Type': 'application/json'}
r = requests.post(url, data=json.dumps(data), headers=headers)

print("Response status code:", r.status_code)
print("Response content:", r.text)

Response status code: 200
Response content: {
  "id": "id_lsDOmPS8y0pdyXsy0qwdPA",
  "outcome": false
}



In [29]:
#import pandas as pd
#print("My numpy version is: ", pd.__version__)

My numpy version is:  2.1.0


In [34]:
import requests

# Define the URL of your Flask application
url = 'http://127.0.0.1:5000/recidivism_result/'  # Update with your actual URL

# Define the observation data
data = {
    "id": None,
    "outcome": True
}

headers = {'Content-Type': 'application/json'}
response = requests.post(url, data=json.dumps(data), headers=headers)

# Check the response status code
if response.status_code == 200:
    try:
        result = response.json()
        observation_id = result.get('id')  # Correct key
        outcome = result.get('outcome')  # Correct key
        predicted_outcome = result.get('predicted_outcome')  # Correct key
        if observation_id is not None and outcome is not None:
            print("Recidivism outcome updated successfully for ID:", observation_id)
            print("True outcome:", outcome)
            print("Predicted outcome:", predicted_outcome)
        else:
            print("Observation ID or outcome not found in the response")
    except ValueError:
        print("Failed to parse JSON response")
else:
    print("Failed to update recidivism outcome:", response.text)


Observation ID or outcome not found in the response


In [12]:
import requests
import json

url = "https://capstone-production-a625.up.railway.app/will_recidivate/"

data = {
    "id": "teste123",
    "name": "John Doe",
    "sex": "Male",
    "dob": "1990-01-01",
    "race": "White",
    "juv_fel_count": 0,
    "juv_misd_count": 1,
    "juv_other_count": 0,
    "priors_count": 2,
    "c_case_number": "case1234",
    "c_charge_degree": "M",
    "c_charge_desc": "Theft",
    "c_offense_date": "2023-01-01",
    "c_arrest_date": "2023-01-02",
    "c_jail_in": "2023-01-03"
}


headers = {'Content-Type': 'application/json'}
r = requests.post(url, data=json.dumps(data), headers=headers)

print("Response status code:", r.status_code)
print("Response content:", r.text)

Response status code: 200
Response content: {
  "id": "teste123",
  "outcome": false
}



In [14]:
def test_recidivism_result():
    url = "https://capstone-production-a625.up.railway.app/recidivism_result/"
    data = {
        "id": "teste1234",
        "outcome": True
    }

    try:
        response = requests.post(url, json=data)
        response.raise_for_status()  # Raise an error for bad status codes
        print(f"Status Code: {response.status_code}")
        print(response.json())
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        if response is not None:
            print(f"Status Code: {response.status_code}")
            print(response.text)
            
test_recidivism_result()

Status Code: 200
{'error': 'Observation ID: "teste1234" does not exist'}


In [36]:
import requests

def test_will_recidivate():
    url = "https://capstone-production-8415.up.railway.app/will_recidivate/"
    data = {
        "id": "a1b2c3",
        "name": "John Doe",
        "sex": "Male",
        "dob": "1990-01-01",
        "race": "Caucasian",
        "juv_fel_count": 2,
        "juv_misd_count": 1,
        "juv_other_count": 0,
        "priors_count": 5,
        "c_case_number": "A123456",
        "c_charge_degree": "F",
        "c_charge_desc": "Burglary",
        "c_offense_date": "2022-01-01",
        "c_arrest_date": "2022-01-02",
        "c_jail_in": "2022-01-03"
    }

    try:
        response = requests.post(url, json=data)
        response.raise_for_status()  # Raise an error for bad status codes
        print(f"Status Code: {response.status_code}")
        print(response.json())
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        if response is not None:
            print(f"Status Code: {response.status_code}")
            print(response.text)

def test_recidivism_result():
    url = "https://capstone-production-8415.up.railway.app/recidivism_result/"
    data = {
        "id": "a1b2c3456",
        "outcome": True
    }

    try:
        response = requests.post(url, json=data)
        response.raise_for_status()  # Raise an error for bad status codes
        print(f"Status Code: {response.status_code}")
        print(response.json())
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        if response is not None:
            print(f"Status Code: {response.status_code}")
            print(response.text)

def list_db_contents():
    url = "https://capstone-production-8415.up.railway.app/list-db-contents"

    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad status codes
        print(f"Status Code: {response.status_code}")
        print(response.json())
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        if response is not None:
            print(f"Status Code: {response.status_code}")
            print(response.text)

# Run the tests
test_will_recidivate()
test_recidivism_result()
list_db_contents()


Status Code: 200
{'error': 'Observation ID: "a1b2c3" already exists', 'id': 'a1b2c3', 'outcome': False}
Status Code: 200
{'error': 'Observation ID: "a1b2c3456" does not exist'}
Status Code: 200
[{'id': 1, 'observation': '{"id": "a1b2c34567", "name": "6cea57c2fb6cbc2a40411135005760f241fffc3e5e67ab99882726431037f908", "sex": "Male", "dob": "1990-01-01", "race": "White", "juv_fel_count": 0, "juv_misd_count": 1, "juv_other_count": 0, "priors_count": 2, "c_case_number": "case1234", "c_charge_degree": "M", "c_charge_desc": "Theft", "c_offense_date": "2023-01-01", "c_arrest_date": "2023-01-02", "c_jail_in": "2023-01-03"}', 'observation_id': 'a1b2c34567', 'outcome': False, 'predicted_outcome': False}, {'id': 2, 'observation': '{"id": "id_KVNBmR0jEq6hJBXN3L-rig", "name": "6cea57c2fb6cbc2a40411135005760f241fffc3e5e67ab99882726431037f908", "sex": "Male", "dob": "1990-01-01", "race": "White", "juv_fel_count": 0, "juv_misd_count": 1, "juv_other_count": 0, "priors_count": 2, "c_case_number": "case12

In [5]:
curl -X POST https://capstone-production-fe80.up.railway.app/will_recidivate/ -d '{"id": "test123",
        "name": "John Doe",
        "sex": "Male",
        "dob": "1990-01-01",
        "race": "Caucasian",
        "juv_fel_count": 2,
        "juv_misd_count": 1,
        "juv_other_count": 0,
        "priors_count": 5,
        "c_case_number": "A123456",
        "c_charge_degree": "F",
        "c_charge_desc": "Burglary",
        "c_offense_date": "2022-01-01",
        "c_arrest_date": "2022-01-02",
        "c_jail_in": "2022-01-03"}' -H "Content-Type:application/json"

SyntaxError: unterminated string literal (detected at line 1) (710772726.py, line 1)

In [ ]:
curl -X POST https://capstone-production-d66f.up.railway.app/recidivism_result -d '{
         "id": "123456abc",
        "outcome": true
         }' -H "Content-Type:application/json"